In [1]:
import multiprocessing.pool
import functools
import requests
import pandas as pd
from dateutil.relativedelta import relativedelta
import warnings
import os
import shutil
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import numpy as np
import time
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
import ag_email_utils

In [11]:
start_time = time.time()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('expand_frame_repr', False)
pd.options.display.float_format = '{:,.2f}'.format

############### SPECIFY (note: JODI runs can be amended on line 2613)##############################################
backcast_months = 4 # forecast back dates 4 months from today
graph_historic_years = 5 # graph data from 5 yrs back
forecast_end_date = pd.to_datetime((datetime.now()+relativedelta(years=1)).strftime('%Y-12-01')) # forecast till end of next year.
months_ahead = len(pd.date_range(start=pd.to_datetime((datetime.now()-relativedelta(months=backcast_months)).strftime('%Y-%m-01')), end=forecast_end_date, freq='MS'))
correlation_months_spec = 25 # months used for correlation
timeout_sec= 30 #time (seconds) allowed for query to run before error message
long_timeout_sec=120 #time (seconds) allowed for query to run before error message
Pdate_days_before=0 #0 is today and 1 is yesterday
correlation_utilisation_upper_limit = 1.5 # upper limit to discard utilisation rates as anomaly

IIR_Countries = ['Austria','Belgium','Denmark','Finland','France','Germany','Greece','Ireland','Italy','Lithuania','Netherlands','Norway','Portugal','Spain','Sweden','United Kingdom','Turkey','Czech Republic','Hungary','Poland','Slovakia','Switzerland']
NWE_Countries = ['Belgium', 'Denmark', 'Finland', 'Germany', 'Ireland','Lithuania', 'Netherlands', 'Norway', 'Sweden', 'United Kingdom','France NWE']
Med_Countries = ['Austria', 'Czech Republic', 'Greece', 'Hungary', 'Italy', 'Poland', 'Portugal', 'Slovakia', 'Spain','Switzerland', 'Turkey', 'France Med']
Med_Countries_for_table = ['Austria', 'Greece', 'Italy', 'Portugal', 'Spain','Switzerland', 'Turkey', 'France Med']

#Make sure IEA and JODI country list mataches IIR country if sorted alphabetically
IEA_Countries = ['AUSTRIA','BELGIUM','CZECH','DENMARK','FINLAND','FRANCE','GERMANY','GREECE','HUNGARY','IRELAND','ITALY','LITHUANIA','NETHLAND','NORWAY','POLAND','PORTUGAL','SLOVAKIA','SPAIN','SWEDEN','SWITLAND','TURKEY','UK']
JODI_Countries = ['AT','BE','CZ','DK','FI','FR','DE','GR','HU','IE','IT','LT','NL','NO','PL','PT','SK','ES','SE','CH','TR','GB']

# folder = r'\\petroineos.local\dfs\Department Private Folders\Analysis Department\Projects\Europe Refinery Run\Archive'
folder = r'C:\Users\syedahmad\Documents\refinery_runs_model\runs_model'
sub_folder = folder + "\\" + time.strftime("%Y-%m-%d")
folder_diagrams = sub_folder + "\\" + "Diagrams"
folder_scrape = folder + "\\" + "Scrape"
folder_results = folder + "\\" + time.strftime("%Y-%m-%d") + "\\" + "Results"

if os.path.isdir(sub_folder):
    shutil.rmtree(sub_folder)
else:
    pass

os.makedirs(folder_diagrams)
# os.makedirs(folder_scrape)
os.makedirs(folder_results)

address_for_error_emails = "Syed.Ahmad@petroineos.com"
sender_address_for_auto_email = "Syed.Ahmad@petroineos.com"
recipient_address_for_auto_email = "Syed.Ahmad@petroineos.com"
############################################################################


start_date = (datetime.now()-relativedelta(months=backcast_months)).strftime('%Y-%m-01')
start_date = pd.Timestamp(start_date)
end_date = start_date + relativedelta(months=months_ahead)

correlation_end_date= start_date
correlation_start_date=correlation_end_date - relativedelta(months=correlation_months_spec)

Pdate_days_before=str(Pdate_days_before+1)

#get jodi data start date which ever is earlier (1st Jan 2 years ago or 2 years 5 months from today)
graph_start_date = (datetime.now()-relativedelta(years=graph_historic_years)).strftime('%Y-01-01')
if graph_start_date<correlation_start_date.strftime('%Y-%m-01'):
    historic_start_date=graph_start_date
else:
    historic_start_date = correlation_start_date

correlation_months = len(pd.date_range(start=historic_start_date, end=correlation_end_date, freq='MS'))-1


forecast_range = pd.DataFrame({'date': pd.date_range(start=start_date, end=end_date, freq='MS')})
correlation_range = pd.DataFrame({'date': pd.date_range(start=historic_start_date, end=correlation_end_date, freq='MS')})
year_range = pd.DataFrame({'date': pd.date_range(start=datetime.now().date().replace(month=1,day=1), end=datetime.now().date().replace(month=12,day=1), freq='MS')})
timeframes = [forecast_range,correlation_range,year_range]

### -------- Helper functions-----------------

In [9]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

def send_error_report(email_from, emails,topic,date):
    sub = "European Refinery Runs Forecast Error" #- (" + datetime.today().strftime(format_datetime) + ")"
    body = "<font size='+1'><b>Error on "+topic+" - "+date+" .</b></font>" \
           + "<br>" \

    img = None
    email_sender = ag_email_utils.AgEmailHelper()
    email_sender.send_mail_with_embedded_images(email_from, emails, sub, body,img)


def list_to_string(locations):
    string = ""
    for location in locations:
        string += "'"+location+"', "
    string = string[:-2]
    return string

def country():
    query = ''' 
SELECT DISTINCT[country]
FROM   [refineryunit]
WHERE  country IN ( ''' + list_to_string(IIR_Countries) + ''' )
ORDER  BY country 
    '''
    df = ag_get_data(query)
    return df


def nameplate(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m')
    #end_date = pd.Timestamp(timeframes['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
WITH temp AS
(
         SELECT   [refineryunit].[country]                           AS country,
                  Sum([RefineryUnitCapacityMonthly].[Capacity])/1000 AS cap
         FROM     [Refinery].[RefineryUnitCapacityMonthly],
                  [refineryunit],[RefineryUnitCapacitySchedule]
         WHERE    [RefineryUnitCapacityMonthly].[Unit_ID] = [refineryunit].[unit_id]
         AND       [RefineryUnitCapacityMonthly].[Unit_ID] = [RefineryUnitCapacitySchedule].[Unit_ID]
         AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
         AND      [refineryunit].[country] IN (''' + list_to_string(IIR_Countries) + ''')
         AND      [RefineryUnitCapacityMonthly].[DDate] LIKE ''' + "'" + start_date  + "%" + "'" + '''
         AND     NOT( ([RefineryUnit].[U_STATUS] != 'Operational' and [RefineryUnitCapacitySchedule].[EndDate]='9998-12-31'))
         GROUP BY [refineryunit].[country]) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country,
                isnull(temp.cap,0) AS cap
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country

                '''
    df0 = ag_get_data(query)
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
    df0[start_date] = df0['cap']
    df0 = df0.drop(columns=['cap'])
    #df2 = country()
    #df2[start_date]= pd.Series([df1])
    #b = pd.concat([country(), df1], axis=1, join="inner")
    #df1[start_date] = df1['Planned_CAP_OFF']
    #df1 = df1.drop(columns=['Planned_CAP_OFF'])
    return df0


def p_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
 WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC) ,temp AS
(
         SELECT   [refineryunit].[country]                                                                                                           AS country,
                  Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
         FROM     [Refinery].[maintenancedaily],
                  [Refinery].[refineryunit],
                  [Refinery].[refinerymaintenancebestlatest]
         WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
         AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
         AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
         AND      [refineryunit].[country] IN (''' + list_to_string(IIR_Countries) + ''')
         AND      ((
                                    [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                           AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
         AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
         AND      (
                           [refinerymaintenancebestlatest].[DERATE] IS NULL
                  OR       ((
                                             [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                           AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
         AND      [maintenancedaily].[pdate] =
                  (
                           SELECT TOP(1)
                                    [pdate]
                           FROM     temp0
                           ORDER BY [PDATE])
         GROUP BY [refineryunit].[country]) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [Refinery].[refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country           AS country,
                isnull(temp.cap_off,0) AS planned_cap_off
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country
                '''

    @timer(timeout_sec)
    def dff1():
        df1 = ag_get_data(query)
        # df2 = country()
        # df2[start_date]= pd.Series([df1])
        # b = pd.concat([country(), df1], axis=1, join="inner")
        df1[start_date] = df1['planned_cap_off']
        df1 = df1.drop(columns=['planned_cap_off'])
        return df1

    try:
        df1=dff1()
    except multiprocessing.context.TimeoutError:
        try:
            df1 = dff1()
        except multiprocessing.context.TimeoutError:
            print('ERROR ON '+start_date)
            send_error_report(address_for_error_emails, address_for_error_emails,'planned known TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df1 = country()
            df1[start_date] = 'Error'
            df1.replace(np.nan, 0, inplace=True)
    return df1



def u_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
          WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC) ,temp AS
(
         SELECT   [refineryunit].[country]                                                                                                           AS country,
                  Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
         FROM     [Refinery].[maintenancedaily],
                  [Refinery].[refineryunit],
                  [Refinery].[refinerymaintenancebestlatest]
         WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
         AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
         AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
         AND      [refineryunit].[country] IN (''' + list_to_string(IIR_Countries) + ''')
         AND      ((
                                    [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                           AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
         AND      [refinerymaintenancebestlatest].[event_type] = 'Unplanned'
         AND      (
                           [refinerymaintenancebestlatest].[DERATE] IS NULL
                  OR       ((
                                             [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                    OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                           AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
         AND      [maintenancedaily].[pdate] =
                  (
                           SELECT TOP(1)
                                    [pdate]
                           FROM     temp0
                           ORDER BY [PDATE])
         GROUP BY [refineryunit].[country]) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [Refinery].[refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country           AS country,
                isnull(temp.cap_off,0) AS planned_cap_off
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country


                '''
    @timer(timeout_sec)
    def dff2():
        df2 = ag_get_data(query)
        df2[start_date] = df2['planned_cap_off']
        df2 = df2.drop(columns=['planned_cap_off'])
        return df2

    try:
        df2=dff2()
    except multiprocessing.context.TimeoutError:
        print('ERROR ON ' + start_date)
        try:
            df2 = dff2()
        except multiprocessing.context.TimeoutError:
            print('ERROR ON ' + start_date)
            send_error_report(address_for_error_emails, address_for_error_emails,'unplanned known outage',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df2 = country()
            df2[start_date] = 'Error'
#             df2[start_date] = 0 errormessagin breaking floats
            df2.replace(np.nan, 0, inplace=True)
    return df2

def p_a(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    MONTH = str(datetime.strptime(start_date, "%Y-%m-%d").month)
    query = '''
          WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC) ,temp AS
(
         SELECT   country,
                  (Sum(asset_sums))/
                  (
                         SELECT day (eomonth(''' + "'" + start_date + "'" + ''')) * 5) AS historic
         FROM     (
                           SELECT   [refineryunit].[country],
                                    [maintenancedaily].[OUTAGE_DATE],
                                    sum([maintenancedaily].[cap_offline])/1000 AS asset_sums
                           FROM     [Refinery].[maintenancedaily],
                                    [Refinery].[refineryunit],
                                    [Refinery].[refinerymaintenancebestlatest]
                           WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
                           AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
                           AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
                           AND      [refineryunit].[country]IN (''' + list_to_string(IIR_Countries) + ''')
                           AND      [maintenancedaily].[outage_date]<
                                    (
                                           SELECT dateadd(day,1,eomonth(''' + "'" + start_date + "'" + ''',-1)))
                           AND      (
                                             month([maintenancedaily].[outage_date]) = ''' + MONTH + ''' )
                           AND      (
                                             year([maintenancedaily].[outage_date]) >= (year( dateadd(year,-5,''' + "'" + start_date + "'" + '''))) )
                           AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
                           AND      (
                                             [refinerymaintenancebestlatest].[DERATE] IS NULL
                                    OR       ((
                                                               [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                                      OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                                      OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                                      OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                             AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
                           AND      [maintenancedaily].[pdate] =
                                    (
                                             SELECT TOP(1)
                                                      [pdate]
                                             FROM     temp0
                                             ORDER BY [PDATE])
                           GROUP BY [refineryunit].[country],
                                    [refineryunit].[utype_desc],
                                    [maintenancedaily].[OUTAGE_DATE] ) AS inner_query
         GROUP BY country) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [Refinery].[refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country            AS country,
                isnull(temp.historic,0) AS historic_cap_off
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country




                '''
    @timer(long_timeout_sec)
    def dff3():
        df3 = ag_get_data(query)
        df3[start_date] = df3['historic_cap_off']
        df3 = df3.drop(columns=['historic_cap_off'])
        return df3

    try:
        df3=dff3()
    except multiprocessing.context.TimeoutError:
        print('ERROR ON ' + start_date)
        try:
            df3 = dff3()
        except multiprocessing.context.TimeoutError:
            print('ERROR ON ' + start_date)
            send_error_report(address_for_error_emails, address_for_error_emails,'planned 5yr average TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df3 = country()
            df3[start_date] = 'Error'
            df3.replace(np.nan, 0, inplace=True)
    return df3


def u_a(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    MONTH = str(datetime.strptime(start_date, "%Y-%m-%d").month)
    query = '''
WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC) ,temp AS
(
         SELECT   country,
                  (Sum(asset_sums))/
                  (
                         SELECT day (eomonth(''' + "'" + start_date + "'" + ''')) * 5) AS historic
         FROM     (
                           SELECT   [refineryunit].[country],
                                    [maintenancedaily].[OUTAGE_DATE],
                                    sum([maintenancedaily].[cap_offline])/1000 AS asset_sums
                           FROM     [Refinery].[maintenancedaily],
                                    [Refinery].[refineryunit],
                                    [Refinery].[refinerymaintenancebestlatest]
                           WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
                           AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
                           AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
                           AND      [refineryunit].[country]IN (''' + list_to_string(IIR_Countries) + ''')
                           AND      [maintenancedaily].[outage_date]<
                                    (
                                           SELECT dateadd(day,1,eomonth(''' + "'" + start_date + "'" + ''',-1)))
                           AND      (
                                             month([maintenancedaily].[outage_date]) = ''' + MONTH + ''' )
                           AND      (
                                             year([maintenancedaily].[outage_date]) >= (year( dateadd(year,-5,''' + "'" + start_date + "'" + '''))) )
                           AND      [refinerymaintenancebestlatest].[event_type] = 'Unplanned'
                           AND      NOT(((
                                                               year([maintenancedaily].[outage_date]) = '2020')
                                             AND      (
                                                               [refinerymaintenancebestlatest].[DERATE] IS NULL)
                                             AND      (
                                                               [refinerymaintenancebestlatest].[E_CAUSE] = 'Economic')))
                           AND      (
                                             [refinerymaintenancebestlatest].[DERATE] IS NULL
                                    OR       ((
                                                               [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                                      OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                                      OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                                      OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                             AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
                           AND      [maintenancedaily].[pdate] =
                                    (
                                             SELECT TOP(1)
                                                      [pdate]
                                             FROM     temp0
                                             ORDER BY [PDATE])
                           GROUP BY [refineryunit].[country],
                                    [refineryunit].[utype_desc],
                                    [maintenancedaily].[OUTAGE_DATE] ) AS inner_query
         GROUP BY country) , ctry AS
(
                SELECT DISTINCT [country]
                FROM            [Refinery].[refineryunit]
                WHERE           country IN(''' + list_to_string(IIR_Countries) + '''))
SELECT          ctry.country            AS country,
                isnull(temp.historic,0) AS historic_cap_off
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.country = temp.country
ORDER BY        ctry.country



                '''
    @timer(long_timeout_sec)
    def dff4():
        df4 = ag_get_data(query)
        df4[start_date] = df4['historic_cap_off']
        df4 = df4.drop(columns=['historic_cap_off'])
        return df4

    try:
        df4=dff4()
    except multiprocessing.context.TimeoutError:
        print('ERROR ON ' + start_date)
        try:
            df4 = dff4()
        except multiprocessing.context.TimeoutError:
            print('ERROR ON ' + start_date)
            send_error_report(address_for_error_emails, address_for_error_emails,'unplanned 5yr average outage',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df4 = country()
            df4[start_date] = 'Error'
            df4.replace(np.nan, 0, inplace=True)
    return df4

def iea(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    MONTH = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%b%Y')
    query = '''
           WITH temp AS
(
       SELECT [Country],
              [Value]/ ( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') ) AS run_rate
       FROM   [OIL].[dbo].[IeaCrudeBal]
       WHERE  [ProductType]= 'CRUDEOIL'
       AND    [Flow]= 'REFINOBSB'
       AND    [Country] IN (''' + list_to_string(IEA_Countries) + ''')
       AND    [Month] LIKE ''' + "'" + MONTH + "'" + '''
       AND    [PDate] =
              (
                     SELECT max([IeaCrudeBal].[PDate])
                     FROM   [IeaCrudeBal])) , ctry AS
(
                SELECT DISTINCT [OIL].[dbo].[IeaOilImpor].[SourceCountry]
                FROM            [OIL].[dbo].[IeaOilImpor]
                WHERE           [SourceCountry] IN(''' + list_to_string(IEA_Countries) + '''))
SELECT          ctry.[SourceCountry]    AS iea_code,
                isnull(temp.run_rate,0) AS iea
FROM            ctry
LEFT OUTER JOIN temp
ON              ctry.[SourceCountry] = temp.country
ORDER BY        ctry.[SourceCountry]
                '''
    if pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m') >= pd.Timestamp((datetime.now() - relativedelta(months=1))).strftime('%Y-%m'):
        df5 = pd.DataFrame({'iea_code': IEA_Countries})
        df5[start_date] = 0.0
        return df5
    else:
        df5 = ag_get_data(query)
        df5[start_date] = df5['iea']
        df5 = df5.drop(columns=['iea'])
        return df5
    return df5

def jodi(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    query = '''
WITH temp2 AS
(
       SELECT [UNIT_MEASURE],
              [ENERGY_PRODUCT],
              [FLOW_BREAKDOWN],
              [TIME_PERIOD],
              [PDate],
              [REF_AREA],
              [OBS_VALUE]
       FROM   [OIL].[dbo].[JodiCrude]
       WHERE  [FLOW_BREAKDOWN] = 'REFINOBS'
       AND    [ENERGY_PRODUCT] ='CRUDEOIL'
       AND    [REF_AREA] IN (''' + list_to_string(JODI_Countries) + ''')
       AND    [UNIT_MEASURE]='KBD'
       AND    [TIME_PERIOD]=''' + "'" + start_date + "'" + '''
       AND    pdate IN
                        (
                        SELECT DISTINCT [PDate]
                        FROM            [OIL].[dbo].[JodiCrude]
                        WHERE           [FLOW_BREAKDOWN] = 'REFINOBS'
                        AND             [ENERGY_PRODUCT] ='CRUDEOIL'
                        AND             [UNIT_MEASURE]='KBD'
                        AND             [TIME_PERIOD]=''' + "'" + start_date + "'" + ''' )) ,temp1 AS
(
           SELECT     temp2.[REF_AREA],
                      temp2.[OBS_VALUE] AS run_rate
           FROM       temp2
           INNER JOIN
                      (
                               SELECT   max([PDate])AS pdate,
                                        [REF_AREA],
                                        [FLOW_BREAKDOWN],
                                        [ENERGY_PRODUCT],
                                        [UNIT_MEASURE],
                                        [TIME_PERIOD]
                               FROM     temp2
                               WHERE    [FLOW_BREAKDOWN] = 'REFINOBS'
                               AND      [ENERGY_PRODUCT] ='CRUDEOIL'
                               AND      [REF_AREA] IN (''' + list_to_string(JODI_Countries) + ''')
                               AND      [UNIT_MEASURE]='KBD'
                               AND      [TIME_PERIOD]=''' + "'" + start_date + "'" + '''
                               AND      [OBS_VALUE] <> '0'
                               GROUP BY ref_area,
                                        [FLOW_BREAKDOWN],
                                        [ENERGY_PRODUCT],
                                        [UNIT_MEASURE],
                                        [TIME_PERIOD]) AS temp
           ON         temp2.ref_area =temp.ref_area
           AND        temp2.[PDate] =temp.[PDate]
           AND        temp2.[TIME_PERIOD] =temp.[TIME_PERIOD]
           AND        temp2.[FLOW_BREAKDOWN] =temp.[FLOW_BREAKDOWN]
           AND        temp2.[ENERGY_PRODUCT] =temp.[ENERGY_PRODUCT]
           AND        temp2.[UNIT_MEASURE] =temp.[UNIT_MEASURE]) , ctry AS
(
                SELECT DISTINCT [OIL].[dbo].[JodiCrude].[REF_AREA]
                FROM            [OIL].[dbo].[JodiCrude]
                WHERE           [REF_AREA] IN(''' + list_to_string(JODI_Countries) + '''))
SELECT          ctry.[REF_AREA]          AS jodi_code,
                isnull(temp1.run_rate,0) AS jodi
FROM            ctry
LEFT OUTER JOIN temp1
ON              ctry.[REF_AREA] = temp1.[REF_AREA]
ORDER BY        ctry.[REF_AREA]
                '''
    if pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m')>= pd.Timestamp((datetime.now()-relativedelta(months=1))).strftime('%Y-%m'):
        df6=pd.DataFrame({'jodi_code':JODI_Countries})
        df6[start_date] = 0.0
        return df6
    else:
        df6 = ag_get_data(query)
        df6[start_date] = df6['jodi']
        df6 = df6.drop(columns=['jodi'])
        return df6
    return df6

def FMED_nameplate(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m')
    #end_date = pd.Timestamp(timeframes['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
          SELECT   Sum([RefineryUnitCapacityMonthly].[Capacity])/1000 AS cap
FROM     [Refinery].[RefineryUnitCapacityMonthly],
         [Refinery].[refineryunit],
         [Refinery].[RefineryPlant],[RefineryUnitCapacitySchedule]
WHERE    [RefineryUnitCapacityMonthly].[Unit_ID] = [refineryunit].[unit_id]
AND       [RefineryUnitCapacityMonthly].[Unit_ID] = [RefineryUnitCapacitySchedule].[Unit_ID]
AND      [RefineryPlant].[PLANT_ID] = [refineryunit].[PLANT_ID]
AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
AND      [refineryunit].[country] = 'France'
AND      [RefineryPlant].[SUB_REGION] = 'Med'
AND      [RefineryUnitCapacityMonthly].[DDate] LIKE ''' + "'" + start_date + "%" + "'" + '''
AND     NOT( ([RefineryUnit].[U_STATUS] != 'Operational' and [RefineryUnitCapacitySchedule].[EndDate]='9998-12-31'))
GROUP BY [refineryunit].[country]
                '''
    df7 = ag_get_data(query)
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
    df7[start_date] = df7['cap']
    df7 = df7.drop(columns=['cap'])
    return df7

def FNWE_nameplate(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m')
    #end_date = pd.Timestamp(timeframes['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
             SELECT   Sum([RefineryUnitCapacityMonthly].[Capacity])/1000 AS cap
FROM     [Refinery].[RefineryUnitCapacityMonthly],
         [Refinery].[refineryunit],
         [Refinery].[RefineryPlant],[RefineryUnitCapacitySchedule]
WHERE    [RefineryUnitCapacityMonthly].[Unit_ID] = [refineryunit].[unit_id]
AND       [RefineryUnitCapacityMonthly].[Unit_ID] = [RefineryUnitCapacitySchedule].[Unit_ID]
AND      [RefineryPlant].[PLANT_ID] = [refineryunit].[PLANT_ID]
AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
AND      [refineryunit].[country] = 'France'
AND      [RefineryPlant].[SUB_REGION] = 'NWE'
AND      [RefineryUnitCapacityMonthly].[DDate] LIKE ''' + "'" + start_date + "%" + "'" + '''
AND     NOT( ([RefineryUnit].[U_STATUS] != 'Operational' and [RefineryUnitCapacitySchedule].[EndDate]='9998-12-31'))
GROUP BY [refineryunit].[country]
                    '''
    df8 = ag_get_data(query)
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
    df8[start_date] = df8['cap']
    df8 = df8.drop(columns=['cap'])
    return df8

def FMED_p_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
      WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT   Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
FROM     [Refinery].[maintenancedaily],
         [Refinery].[refineryunit],
         [Refinery].[refinerymaintenancebestlatest],
         [Refinery].[refineryplant]
WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
AND      [refineryplant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
AND      [refineryunit].[country] = 'France'
AND      [refineryplant].[SUB_REGION] = 'Med'
AND      ((
                           [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                  AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
AND      (
                  [refinerymaintenancebestlatest].[DERATE] IS NULL
         OR       ((
                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
AND      [maintenancedaily].[pdate] =
         (
                  SELECT TOP(1)
                           [pdate]
                  FROM     temp0
                  ORDER BY [PDATE])
GROUP BY [refineryunit].[country],
         [refineryunit].[utype_desc]
ORDER BY [refineryunit].[country],
         [refineryunit].[utype_desc]
                '''


    @timer(timeout_sec)
    def dff9():
        df9 = ag_get_data(query)
        if df9.empty:
            df9['cap_off'] = [0]
            df9[start_date] = df9['cap_off']
            df9 = df9.drop(columns=['cap_off'])
            df9.replace(np.nan, 0, inplace=True)
            return df9
        else:
            df9[start_date] = df9['cap_off']
            df9 = df9.drop(columns=['cap_off'])
            df9.replace(np.nan, 0, inplace=True)
            return df9


    try:
        df9=dff9()
    except multiprocessing.context.TimeoutError:
        try:
            df9 = dff9()
        except multiprocessing.context.TimeoutError:
            df9 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FMED planned known TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df9[start_date] = df9['cap_off']
            df9 = df9.drop(columns=['cap_off'])
            df9.replace(np.nan, 0, inplace=True)
    return df9


def FNWE_p_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
      WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT   Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
FROM     [Refinery].[maintenancedaily],
         [Refinery].[refineryunit],
         [Refinery].[refinerymaintenancebestlatest],
         [Refinery].[refineryplant]
WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
AND      [refineryplant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
AND      [refineryunit].[country] = 'France'
AND      [refineryplant].[SUB_REGION] = 'NWE'
AND      ((
                           [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                  AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
AND      (
                  [refinerymaintenancebestlatest].[DERATE] IS NULL
         OR       ((
                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
AND      [maintenancedaily].[pdate] =
         (
                  SELECT TOP(1)
                           [pdate]
                  FROM     temp0
                  ORDER BY [PDATE])
GROUP BY [refineryunit].[country],
         [refineryunit].[utype_desc]
ORDER BY [refineryunit].[country],
         [refineryunit].[utype_desc]
                '''
    @timer(timeout_sec)
    def dff10():
        df10 = ag_get_data(query)
        if df10.empty:
            df10['cap_off'] = [0]
            df10[start_date] = df10['cap_off']
            df10 = df10.drop(columns=['cap_off'])
            df10.replace(np.nan, 0, inplace=True)
            return df10
        else:
            df10[start_date] = df10['cap_off']
            df10 = df10.drop(columns=['cap_off'])
            df10.replace(np.nan, 0, inplace=True)
            return df10


    try:
        df10=dff10()
    except multiprocessing.context.TimeoutError:
        try:
            df10 = dff10()
        except multiprocessing.context.TimeoutError:
            df10 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FNWE planned known TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df10[start_date] = df10['cap_off']
            df10 = df10.drop(columns=['cap_off'])
            df10.replace(np.nan, 0, inplace=True)
    return df10

def FMED_u_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
       WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT   Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
FROM     [Refinery].[maintenancedaily],
         [Refinery].[refineryunit],
         [Refinery].[refinerymaintenancebestlatest],
         [Refinery].[refineryplant]
WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
AND      [refineryplant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
AND      [refineryunit].[country] = 'France'
AND      [refineryplant].[SUB_REGION] = 'Med'
AND      ((
                           [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                  AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
AND      [refinerymaintenancebestlatest].[event_type] = 'Unplanned'
AND      (
                  [refinerymaintenancebestlatest].[DERATE] IS NULL
         OR       ((
                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
AND      [maintenancedaily].[pdate] =
         (
                  SELECT TOP(1)
                           [pdate]
                  FROM     temp0
                  ORDER BY [PDATE])
GROUP BY [refineryunit].[country],
         [refineryunit].[utype_desc]
ORDER BY [refineryunit].[country],
         [refineryunit].[utype_desc]
                '''

    @timer(timeout_sec)
    def dff11():
        df11 = ag_get_data(query)
        if df11.empty:
            df11['cap_off'] = [0]
            df11[start_date] = df11['cap_off']
            df11 = df11.drop(columns=['cap_off'])
            df11.replace(np.nan, 0, inplace=True)
            return df11
        else:
            df11[start_date] = df11['cap_off']
            df11 = df11.drop(columns=['cap_off'])
            df11.replace(np.nan, 0, inplace=True)
            return df11
    try:
        df11=dff11()
    except multiprocessing.context.TimeoutError:
        try:
            df11 = dff11()
        except multiprocessing.context.TimeoutError:
            df11 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FMED unplanned known outage',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df11[start_date] = df11['cap_off']
            df11 = df11.drop(columns=['cap_off'])
            df11.replace(np.nan, 0, inplace=True)
    return df11
def FNWE_u_k(j,i,n):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(timeframes[j]['date'].loc[n]).strftime('%Y-%m-%d')
    query = '''
        WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT   Sum([maintenancedaily].[cap_offline])/( datediff(day, ''' + "'" + start_date + "'" + ''', ''' + "'" + end_date + "'" + ''') )/1000 AS cap_off
FROM     [Refinery].[maintenancedaily],
         [Refinery].[refineryunit],
         [Refinery].[refinerymaintenancebestlatest],
         [Refinery].[refineryplant]
WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
AND      [refineryplant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
AND      [refineryunit].[country] = 'France'
AND      [refineryplant].[SUB_REGION] = 'NWE'
AND      ((
                           [maintenancedaily].[outage_date] >= ''' + "'" + start_date + "'" + '''
                  AND      [maintenancedaily].[outage_date] < ''' + "'" + end_date + "'" + ''' ))
AND      [refinerymaintenancebestlatest].[event_type] = 'Unplanned'
AND      (
                  [refinerymaintenancebestlatest].[DERATE] IS NULL
         OR       ((
                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
AND      [maintenancedaily].[pdate] =
         (
                  SELECT TOP(1)
                           [pdate]
                  FROM     temp0
                  ORDER BY [PDATE])
GROUP BY [refineryunit].[country],
         [refineryunit].[utype_desc]
ORDER BY [refineryunit].[country],
         [refineryunit].[utype_desc]
                '''

    @timer(timeout_sec)
    def dff12():
        df12 = ag_get_data(query)
        if df12.empty:
            df12['cap_off'] = [0]
            df12[start_date] = df12['cap_off']
            df12 = df12.drop(columns=['cap_off'])
            df12.replace(np.nan, 0, inplace=True)
            return df12
        else:
            df12[start_date] = df12['cap_off']
            df12 = df12.drop(columns=['cap_off'])
            df12.replace(np.nan, 0, inplace=True)
            return df12
    try:
        df12=dff12()
    except multiprocessing.context.TimeoutError:
        try:
            df12 = dff12()
        except multiprocessing.context.TimeoutError:
            df12 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FNWE unplanned known outage',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df12[start_date] = df12['cap_off']
            df12 = df12.drop(columns=['cap_off'])
            df12.replace(np.nan, 0, inplace=True)
    return df12

def FMED_p_a(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    MONTH = str(datetime.strptime(start_date, "%Y-%m-%d").month)
    query = '''
         WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT (Sum(asset_sums))/
       (
              SELECT day (eomonth(''' + "'" + start_date + "'" + ''')) * 5) AS cap_off
FROM   (
                SELECT   sum([maintenancedaily].[cap_offline])/1000 AS asset_sums
                FROM     [Refinery].[maintenancedaily],
                         [Refinery].[refineryunit],
                         [Refinery].[refinerymaintenancebestlatest],
                         [Refinery].[RefineryPlant]
                WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
                AND      [RefineryPlant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
                AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
                AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
                AND      [refineryunit].[country] = 'France'
                AND      [RefineryPlant].[SUB_REGION] = 'Med'
                AND      [maintenancedaily].[outage_date]<
                         (
                                SELECT dateadd(day,1,eomonth(''' + "'" + start_date + "'" + ''',-1)))
                AND      (
                                  month([maintenancedaily].[outage_date]) = ''' + MONTH + ''' )
                AND      (
                                  year([maintenancedaily].[outage_date]) >= (year( dateadd(year,-5,''' + "'" + start_date + "'" + '''))) )
                AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
                AND      (
                                  [refinerymaintenancebestlatest].[DERATE] IS NULL
                         OR       ((
                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
                AND      [maintenancedaily].[pdate] =
                         (
                                  SELECT TOP(1)
                                           [pdate]
                                  FROM     temp0
                                  ORDER BY [PDATE])
                GROUP BY [refineryunit].[country],
                         [refineryunit].[utype_desc],
                         [maintenancedaily].[OUTAGE_DATE] ) AS inner_query

                '''

    @timer(long_timeout_sec)
    def dff13():
        df13 = ag_get_data(query)
        if df13.empty:
            df13['cap_off'] = [0]
            df13[start_date] = df13['cap_off']
            df13 = df13.drop(columns=['cap_off'])
            df13.replace(np.nan, 0, inplace=True)
            return df13
        else:
            df13[start_date] = df13['cap_off']
            df13 = df13.drop(columns=['cap_off'])
            df13.replace(np.nan, 0, inplace=True)
            return df13
    try:
        df13=dff13()
    except multiprocessing.context.TimeoutError:
        try:
            df13 = dff13()
        except multiprocessing.context.TimeoutError:
            df13 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FMED planned 5yr average TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df13[start_date] = df13['cap_off']
            df13 = df13.drop(columns=['cap_off'])
            df13.replace(np.nan, 0, inplace=True)
    return df13

def FNWE_p_a(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    MONTH = str(datetime.strptime(start_date, "%Y-%m-%d").month)
    query = '''
         WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT (Sum(asset_sums))/
       (
              SELECT day (eomonth(''' + "'" + start_date + "'" + ''')) * 5) AS cap_off
FROM   (
                SELECT   sum([maintenancedaily].[cap_offline])/1000 AS asset_sums
                FROM     [Refinery].[maintenancedaily],
                         [Refinery].[refineryunit],
                         [Refinery].[refinerymaintenancebestlatest],
                         [Refinery].[RefineryPlant]
                WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
                AND      [RefineryPlant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
                AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
                AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
                AND      [refineryunit].[country] = 'France'
                AND      [RefineryPlant].[SUB_REGION] = 'NWE'
                AND      [maintenancedaily].[outage_date]<
                         (
                                SELECT dateadd(day,1,eomonth(''' + "'" + start_date + "'" + ''',-1)))
                AND      (
                                  month([maintenancedaily].[outage_date]) = ''' + MONTH + ''' )
                AND      (
                                  year([maintenancedaily].[outage_date]) >= (year( dateadd(year,-5,''' + "'" + start_date + "'" + '''))) )
                AND      [refinerymaintenancebestlatest].[event_type] = 'Planned'
                AND      (
                                  [refinerymaintenancebestlatest].[DERATE] IS NULL
                         OR       ((
                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
                AND      [maintenancedaily].[pdate] =
                         (
                                  SELECT TOP(1)
                                           [pdate]
                                  FROM     temp0
                                  ORDER BY [PDATE])
                GROUP BY [refineryunit].[country],
                         [refineryunit].[utype_desc],
                         [maintenancedaily].[OUTAGE_DATE] ) AS inner_query

                '''

    @timer(long_timeout_sec)
    def dff14():
        df14 = ag_get_data(query)
        if df14.empty:
            df14['cap_off'] = [0]
            df14[start_date] = df14['cap_off']
            df14 = df14.drop(columns=['cap_off'])
            df14.replace(np.nan, 0, inplace=True)
            return df14
        else:
            df14[start_date] = df14['cap_off']
            df14 = df14.drop(columns=['cap_off'])
            df14.replace(np.nan, 0, inplace=True)
            return df14
    try:
        df14=dff14()
    except multiprocessing.context.TimeoutError:
        try:
            df14 = dff14()
        except multiprocessing.context.TimeoutError:
            df14 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FNWE planned 5yr average TAR',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df14[start_date] = df14['cap_off']
            df14 = df14.drop(columns=['cap_off'])
            df14.replace(np.nan, 0, inplace=True)
    return df14

def FMED_u_a(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    MONTH = str(datetime.strptime(start_date, "%Y-%m-%d").month)
    query = '''
        WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT (Sum(asset_sums))/
       (
              SELECT day (eomonth(''' + "'" + start_date + "'" + ''')) * 5) AS cap_off
FROM   (
                SELECT   sum([maintenancedaily].[cap_offline])/1000 AS asset_sums
                FROM     [Refinery].[maintenancedaily],
                         [Refinery].[refineryunit],
                         [Refinery].[refinerymaintenancebestlatest],
                         [Refinery].[RefineryPlant]
                WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
                AND      [RefineryPlant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
                AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
                AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
                AND      [refineryunit].[country] = 'France'
                AND      [RefineryPlant].[SUB_REGION] = 'Med'
                AND      [maintenancedaily].[outage_date]<
                         (
                                SELECT dateadd(day,1,eomonth(''' + "'" + start_date + "'" + ''',-1)))
                AND      (
                                  month([maintenancedaily].[outage_date]) = ''' + MONTH + ''' )
                AND      (
                                  year([maintenancedaily].[outage_date]) >= (year( dateadd(year,-5,''' + "'" + start_date + "'" + '''))) )
                AND      [refinerymaintenancebestlatest].[event_type] = 'Unplanned'
                AND      NOT(((
                                                    year([maintenancedaily].[outage_date]) = '2020')
                                  AND      (
                                                    [refinerymaintenancebestlatest].[DERATE] IS NULL)
                                  AND      (
                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Economic')))
                AND      (
                                  [refinerymaintenancebestlatest].[DERATE] IS NULL
                         OR       ((
                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
                AND      [maintenancedaily].[pdate] =
                         (
                                  SELECT TOP(1)
                                           [pdate]
                                  FROM     temp0
                                  ORDER BY [PDATE])
                GROUP BY [refineryunit].[country],
                         [refineryunit].[utype_desc],
                         [maintenancedaily].[OUTAGE_DATE] ) AS inner_query


                '''

    @timer(long_timeout_sec)
    def dff15():
        df15 = ag_get_data(query)
        if df15.empty:
            df15['cap_off'] = [0]
            df15[start_date] = df15['cap_off']
            df15 = df15.drop(columns=['cap_off'])
            df15.replace(np.nan, 0, inplace=True)
            return df15
        else:
            df15[start_date] = df15['cap_off']
            df15 = df15.drop(columns=['cap_off'])
            df15.replace(np.nan, 0, inplace=True)
            return df15
    try:
        df15=dff15()
    except multiprocessing.context.TimeoutError:
        try:
            df15 = dff15()
        except multiprocessing.context.TimeoutError:
            df15 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FMED unplanned 5yr average outage',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df15[start_date] = df15['cap_off']
            df15 = df15.drop(columns=['cap_off'])
            df15.replace(np.nan, 0, inplace=True)
    return df15

def FNWE_u_a(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    MONTH = str(datetime.strptime(start_date, "%Y-%m-%d").month)
    query = '''
         WITH temp0 AS
(
                SELECT DISTINCT TOP (''' + Pdate_days_before +''') ([maintenancedaily].[pdate])
                FROM            [Refinery].[maintenancedaily]
                ORDER BY        [PDATE] DESC)
SELECT (Sum(asset_sums))/
       (
              SELECT day (eomonth(''' + "'" + start_date + "'" + ''')) * 5)AS cap_off
FROM   (
                SELECT   sum([maintenancedaily].[cap_offline])/1000 AS asset_sums
                FROM     [Refinery].[maintenancedaily],
                         [Refinery].[refineryunit],
                         [Refinery].[refinerymaintenancebestlatest],
                         [Refinery].[RefineryPlant]
                WHERE    [maintenancedaily].[unit_id] = [refineryunit].[unit_id]
                AND      [RefineryPlant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
                AND      [maintenancedaily].[EVENT_ID] = [refinerymaintenancebestlatest].[EVENT_ID]
                AND      [refineryunit].[utype_desc] = 'Atmospheric Distillation'
                AND      [refineryunit].[country] = 'France'
                AND      [RefineryPlant].[SUB_REGION] = 'NWE'
                AND      [maintenancedaily].[outage_date]<
                         (
                                SELECT dateadd(day,1,eomonth(''' + "'" + start_date + "'" + ''',-1)))
                AND      (
                                  month([maintenancedaily].[outage_date]) = ''' + MONTH + ''' )
                AND      (
                                  year([maintenancedaily].[outage_date]) >= (year( dateadd(year,-5,''' + "'" + start_date + "'" + '''))) )
                AND      [refinerymaintenancebestlatest].[event_type] = 'Unplanned'
                AND      NOT(((
                                                    year([maintenancedaily].[outage_date]) = '2020')
                                  AND      (
                                                    [refinerymaintenancebestlatest].[DERATE] IS NULL)
                                  AND      (
                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Economic')))
                AND      (
                                  [refinerymaintenancebestlatest].[DERATE] IS NULL
                         OR       ((
                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                           OR       [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                  AND      [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
                AND      [maintenancedaily].[pdate] =
                         (
                                  SELECT TOP(1)
                                           [pdate]
                                  FROM     temp0
                                  ORDER BY [PDATE])
                GROUP BY [refineryunit].[country],
                         [refineryunit].[utype_desc],
                         [maintenancedaily].[OUTAGE_DATE] ) AS inner_query


                '''

    @timer(long_timeout_sec)
    def dff16():
        df16 = ag_get_data(query)
        if df16.empty:
            df16['cap_off'] = [0]
            df16[start_date] = df16['cap_off']
            df16 = df16.drop(columns=['cap_off'])
            df16.replace(np.nan, 0, inplace=True)
            return df16
        else:
            df16[start_date] = df16['cap_off']
            df16 = df16.drop(columns=['cap_off'])
            df16.replace(np.nan, 0, inplace=True)
            return df16
    try:
        df16=dff16()
    except multiprocessing.context.TimeoutError:
        try:
            df16 = dff16()
        except multiprocessing.context.TimeoutError:
            df16 = pd.DataFrame({'cap_off': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'FNWE unplanned 5yr average outage',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df16[start_date] = df16['cap_off']
            df16 = df16.drop(columns=['cap_off'])
            df16.replace(np.nan, 0, inplace=True)
    return df16

def margin(j,i,region):
    today = (datetime.now()).strftime('%Y-%m')
    today = pd.Timestamp(today).strftime('%Y-%m')
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m')
    query0 = '''
          SELECT ''' + "'" + region + "'" + ''' AS region,
       avg([''' + region + '''])      AS margin
FROM   [OIL].[dbo].[EuroMargin]
WHERE  [type]= '0'
AND    [ddate] LIKE ''' + "'" + start_date + "-%" + "'" + '''
                '''
    query1 = '''
          WITH cal AS
(
       SELECT [Date]
       FROM   [ANALYSIS].[dbo].[Calendar]
       WHERE  [Date] LIKE ''' + "'" + start_date + "-%" + "'" + ''') ,temp1 AS
(
       SELECT [pdate],
              [''' + region + ''']
       FROM   [OIL].[dbo].[EuroMargin]
       WHERE  [type]= '0'
       AND    [pdate] LIKE ''' + "'" + start_date + "-%" + "'" + ''') ,temp2 AS
(
       SELECT top(1) [''' + region + ''']
       FROM   [OIL].[dbo].[EuroMargin]
       WHERE  ([type]= '1' or [type]= '2') 
       AND    [pdate] LIKE ''' + "'" + start_date + "-%" + "'" + '''
       order by [type],ddate desc ) ,temp3 AS
(
                SELECT          isnull([temp1].[''' + region + '''],
                                (
                                       SELECT ([temp2].[''' + region + '''])
                                       FROM   [temp2])) AS margin
                FROM            cal
                LEFT OUTER JOIN temp1
                ON              [cal].[Date] = [temp1].[pdate])
SELECT ''' + "'" + region + "'" + ''' AS region,
       avg ([margin])                 AS margin
FROM   temp3
                '''
    query2 = '''
              SELECT ''' + "'" + region + "'" + ''' AS region,
       [''' + region + ''']           AS margin
FROM   [OIL].[dbo].[EuroMargin]
WHERE  [type]= '2'
AND    [pdate] LIKE ''' + "'" + start_date + "-%" + "'" + '''
AND    [ddate] =
       (
              SELECT max([EuroMargin].[ddate])
              FROM   [OIL].[dbo].[EuroMargin])
                    '''

    @timer(timeout_sec)
    def dff17():
        if start_date<today:
            df17 = ag_get_data(query0)
            new_start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
            if df17.empty:
                df17['margin'] = [np.nan]
                df17[new_start_date] = df17['margin']
                df17 = df16.drop(columns=['margin'])
                df17.replace(np.nan, 'Not in Database', inplace=True)
                return df17
            else:
                df17[new_start_date] = df17['margin']
                df17 = df17.drop(columns=['margin'])
                df17.replace(np.nan, 'Not in Database', inplace=True)
                return df17
        elif start_date==today:
            df17 = ag_get_data(query1)
            new_start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
            if df17.empty:
                df17['margin'] = [np.nan]
                df17[new_start_date] = df17['margin']
                df17 = df16.drop(columns=['margin'])
                df17.replace(np.nan, 'Not in Database', inplace=True)
                return df17
            else:
                df17[new_start_date] = df17['margin']
                df17 = df17.drop(columns=['margin'])
                df17.replace(np.nan, 'Not in Database', inplace=True)
                return df17
        elif start_date>today:
            df17 = ag_get_data(query2)
            new_start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
            if df17.empty:
                df17['margin'] = [np.nan]
                df17[new_start_date] = df17['margin']
                df17 = df17.drop(columns=['margin'])
                df17.replace(np.nan, 'Not in Database', inplace=True)
                return df17
            else:
                df17[new_start_date] = df17['margin']
                df17 = df17.drop(columns=['margin'])
                df17.replace(np.nan, 'Not in Database', inplace=True)
                return df17

    try:
        df17=dff17()
    except multiprocessing.context.TimeoutError:
        df17=pd.DataFrame({'margin':['Error']})
        send_error_report(address_for_error_emails, address_for_error_emails, 'margins',start_date)
        if os.path.isdir(sub_folder):
            shutil.rmtree(sub_folder)
        else:  ## Show an error ##
            print("Error: %s file not found" % sub_folder)
            new_start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-01')
        df17[new_start_date] = df17['margin']
        df17 = df17.drop(columns=['margin'])
        df17.replace(np.nan, 0, inplace=True)
    return df17

def max_5yr_range(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    LAST_YEAR = pd.Timestamp(pd.to_datetime(timeframes[j]['date'].loc[i]) - relativedelta(years=1)).strftime('%Y-%m-%d')
    DATE = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%m-%d')
    PAST = pd.Timestamp(pd.to_datetime(timeframes[j]['date'].loc[i]) - relativedelta(years=5)).strftime('%Y-%m-%d')
    query = f'''
          SELECT MAX (Run_Rate) as Max_Run FROM (
 SELECT   
       [TIME_PERIOD], SUM ([OBS_VALUE]) AS Run_Rate
  FROM [OIL].[dbo].[JodiCrudeLatest]
  WHERE  [FLOW_BREAKDOWN] = 'REFINOBS'
  AND [ENERGY_PRODUCT] ='CRUDEOIL'
  AND [REF_AREA] IN ({list_to_string(JODI_Countries)})
  AND [UNIT_MEASURE]='KBD'
 AND  [TIME_PERIOD] LIKE '%{DATE}'
 AND [TIME_PERIOD] NOT LIKE '2020-%-%' 
AND ([TIME_PERIOD] BETWEEN '{PAST}' AND '{LAST_YEAR}')
 Group by [TIME_PERIOD]) AS TEMP1
                '''

    @timer(long_timeout_sec)
    def dff18():
        df18 = ag_get_data(query)
        if df18.empty:
            df18['Max_Run'] = [0]
            df18[start_date] = df18['Max_Run']
            df18 = df18.drop(columns=['Max_Run'])
            df18.replace(np.nan, 0, inplace=True)
            return df18
        else:
            df18[start_date] = df18['Max_Run']
            df18 = df18.drop(columns=['Max_Run'])
            df18.replace(np.nan, 0, inplace=True)
            return df18
    try:
        df18=dff18()
    except multiprocessing.context.TimeoutError:
        try:
            df18 = dff18()
        except multiprocessing.context.TimeoutError:
            df18 = pd.DataFrame({'Max_Run': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'JODI max 5 year range',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df18[start_date] = df18['Max_Run']
            df18 = df18.drop(columns=['Max_Run'])
            df18.replace(np.nan, 0, inplace=True)
    return df18

def min_5yr_range(j,i):
    start_date = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%Y-%m-%d')
    LAST_YEAR = (pd.to_datetime(timeframes[j]['date'].loc[i]) - relativedelta(years=1)).strftime('%Y-%m-%d')
    DATE = pd.Timestamp(timeframes[j]['date'].loc[i]).strftime('%m-%d')
    PAST = (pd.to_datetime(timeframes[j]['date'].loc[i]) - relativedelta(years=5)).strftime('%Y-%m-%d')
    query = f'''
          SELECT MIN (Run_Rate) as Min_Run FROM (
 SELECT   
       [TIME_PERIOD], SUM ([OBS_VALUE]) AS Run_Rate
  FROM [OIL].[dbo].[JodiCrudeLatest]
  WHERE  [FLOW_BREAKDOWN] = 'REFINOBS'
  AND [ENERGY_PRODUCT] ='CRUDEOIL'
  AND [REF_AREA] IN ({list_to_string(JODI_Countries)})
  AND [UNIT_MEASURE]='KBD'
 AND  [TIME_PERIOD] LIKE '%{DATE}'
 AND [TIME_PERIOD] NOT LIKE '2020-%-%' 
AND ([TIME_PERIOD] BETWEEN '{PAST}' AND '{LAST_YEAR}')
 Group by [TIME_PERIOD]) AS TEMP1
                '''

    @timer(long_timeout_sec)
    def dff19():
        df19 = ag_get_data(query)
        if df19.empty:
            df19['Min_Run'] = [0]
            df19[start_date] = df19['Min_Run']
            df19 = df19.drop(columns=['Min_Run'])
            df19.replace(np.nan, 0, inplace=True)
            return df19
        else:
            df19[start_date] = df19['Min_Run']
            df19 = df19.drop(columns=['Min_Run'])
            df19.replace(np.nan, 0, inplace=True)
            return df19
    try:
        df19=dff19()
    except multiprocessing.context.TimeoutError:
        try:
            df19 = dff19()
        except multiprocessing.context.TimeoutError:
            df19 = pd.DataFrame({'Min_Run': ['Error']})
            send_error_report(address_for_error_emails, address_for_error_emails,'JODI min 5 year range',start_date)
            if os.path.isdir(sub_folder):
                shutil.rmtree(sub_folder)
            else:  ## Show an error ##
                print("Error: %s file not found" % sub_folder)
            df19[start_date] = df19['Min_Run']
            df19 = df19.drop(columns=['Min_Run'])
            df19.replace(np.nan, 0, inplace=True)
    return df19

def get_runs_forecast():
    query = '''
        SELECT [PDATE]
      ,[COUNTRY]
      ,[WORLD_REG]
      ,[SUB_REGION]
      ,[FORECAST_DATE]
      ,[UNITS]
      ,[FORECAST_RUNS]
      ,[FORECAST_UTILISATION]
      ,[FORECAST_PLANNED_OFF]
      ,[FORECAST_UNPLANNED_OFF]
      ,[FORECAST_AVAILABLE_CAPACITY]
  FROM [OIL].[Refinery].[RefineryCountryRunsForecast]
  where PDATE = (select TOP (1)pdate from [OIL].[Refinery].[RefineryCountryRunsForecast] WHERE PDATE > dateadd(day,-30,getdate()) ORDER BY PDATE)
        '''
    df20 = ag_get_data(query)
    #df['PDATE'] = (pd.to_datetime(df['PDATE'])).dt.strftime("%Y-%m")
    #df['FORECAST_DATE'] = (pd.to_datetime(df['FORECAST_DATE'])).dt.strftime("%Y-%m")
    df20_pivot_runs = pd.pivot_table(df20, values='FORECAST_RUNS', index=['COUNTRY'], columns=['FORECAST_DATE'])
    return df20_pivot_runs


def timer(max_timeout):
    """Timeout decorator, parameter in seconds."""
    def timeout_decorator(item):
        """Wrap the original function."""
        @functools.wraps(item)
        def func_wrapper(*args, **kwargs):
            """Closure for function."""
            pool = multiprocessing.pool.ThreadPool(processes=1)
            async_result = pool.apply_async(item, args, kwargs)
            # raises a TimeoutError if execution exceeds max_timeout
            return async_result.get(max_timeout)
        return func_wrapper
    return timeout_decorator

def raw_data_margin_table():
    #GETTING MARGIN TABLE
    region=['nwecomplex','medcomplex']
    nwe_margin = pd.DataFrame({'region':[region[0]]})
    for i in zip(range(0, months_ahead)):
        nwe_margin = pd.merge(nwe_margin, margin(0,i,region[0]), how="inner", on='region')
    #print(nwe_margin)

    med_margin = pd.DataFrame({'region':[region[1]]})
    for i in zip(range(0, months_ahead)):
        med_margin = pd.merge(med_margin, margin(0, i, region[1]), how="inner", on='region')
    #print(med_margin)

    margin_table= pd.concat([nwe_margin,med_margin])
    return margin_table

def raw_data_nameplate_cap():
    #GETTING NAMEPLATE CAPACITY TABLE
    nameplate_cap = country()
    for i in zip(range(0,months_ahead)):
        nameplate_cap = pd.merge(nameplate_cap, nameplate(0,i), how="inner", on='country')

    FMED_nameplate_cap =pd.DataFrame({'country':['France Med']})
    for i in zip(range(0,months_ahead)):
        FMED_nameplate_cap = pd.concat([FMED_nameplate_cap, FMED_nameplate(0,i)],axis=1, join="inner")

    FNWE_nameplate_cap =pd.DataFrame({'country':['France NWE']})
    for i in zip(range(0,months_ahead)):
        FNWE_nameplate_cap = pd.concat([FNWE_nameplate_cap, FNWE_nameplate(0,i)],axis=1, join="inner")

    nameplate_cap= pd.concat([nameplate_cap,FMED_nameplate_cap,FNWE_nameplate_cap])
    #print(FMED_p_k(10,11))
    #sys.exit()
    return nameplate_cap

def raw_data_planned_known():
    #GETTING PLANNED KNOWN MAINTENANCE TABLE
    df_planned_known =country()
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        df_planned_known = pd.merge(df_planned_known, p_k(0,i,n), how="inner", on='country')
    #print(df_planned_known)
    df_FMED_planned_known =pd.DataFrame({'country':['France Med']})
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        df_FMED_planned_known = pd.concat([df_FMED_planned_known, FMED_p_k(0,i,n)],axis=1, join="inner")
    #print(df_FMED_planned_known)
    df_FNWE_planned_known =pd.DataFrame({'country':['France NWE']})
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        df_FNWE_planned_known = pd.concat([df_FNWE_planned_known, FNWE_p_k(0,i,n)],axis=1, join="inner")
    #print(df_FNWE_planned_known)
    df_planned_known= pd.concat([df_planned_known,df_FMED_planned_known,df_FNWE_planned_known])
    df_planned_known.reset_index(inplace=True,drop=True)
    return df_planned_known

def raw_data_unplanned_known():
    #GETTING UNPLANNED KNOWN MAINTENANCE TABLE
    df_unplanned_known =country()
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        df_unplanned_known = pd.merge(df_unplanned_known, u_k(0,i,n), how="inner", on='country')
    #print(df_unplanned_known)
    df_FMED_unplanned_known =pd.DataFrame({'country':['France Med']})
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        df_FMED_unplanned_known = pd.concat([df_FMED_unplanned_known, FMED_u_k(0,i,n)],axis=1, join="inner")
    #print(df_FMED_unplanned_known)
    df_FNWE_unplanned_known =pd.DataFrame({'country':['France NWE']})
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        df_FNWE_unplanned_known = pd.concat([df_FNWE_unplanned_known, FNWE_u_k(0,i,n)],axis=1, join="inner")
    #print(df_FNWE_unplanned_known)
    df_unplanned_known= pd.concat([df_unplanned_known,df_FMED_unplanned_known,df_FNWE_unplanned_known])
    return df_unplanned_known

def raw_data_planned_avg():
    #GETTING PLANNED AVERAGE MAINTENANCE TABLE
    df_planned_avg =country()
    for i in zip(range(0, months_ahead)):
        df_planned_avg = pd.merge(df_planned_avg, p_a(0,i), how="inner", on='country')
    print(df_planned_avg)
    df_FMED_planned_avg =pd.DataFrame({'country':['France Med']})
    for i in zip(range(0,months_ahead)):
        df_FMED_planned_avg = pd.concat([df_FMED_planned_avg, FMED_p_a(0,i)],axis=1, join="inner")
    print(df_FMED_planned_avg)
    df_FNWE_planned_avg =pd.DataFrame({'country':['France NWE']})
    for i in zip(range(0,months_ahead)):
        df_FNWE_planned_avg = pd.concat([df_FNWE_planned_avg, FNWE_p_a(0,i)],axis=1, join="inner")
    print(df_FNWE_planned_avg)
    df_planned_avg= pd.concat([df_planned_avg,df_FMED_planned_avg,df_FNWE_planned_avg])
    return df_planned_avg

def raw_data_unplanned_avg():
    #GETTING UNPLANNED AVERAGE MAINTENANCE TABLE
    df_unplanned_avg =country()
    for i in zip(range(0, months_ahead)):
        df_unplanned_avg = pd.merge(df_unplanned_avg, u_a(0,i), how="inner", on='country')

    df_FMED_unplanned_avg =pd.DataFrame({'country':['France Med']})
    for i in zip(range(0,months_ahead)):
        df_FMED_unplanned_avg = pd.concat([df_FMED_unplanned_avg, FMED_u_a(0,i)],axis=1, join="inner")

    df_FNWE_unplanned_avg =pd.DataFrame({'country':['France NWE']})
    for i in zip(range(0,months_ahead)):
        df_FNWE_unplanned_avg = pd.concat([df_FNWE_unplanned_avg, FNWE_u_a(0,i)],axis=1, join="inner")

    df_unplanned_avg= pd.concat([df_unplanned_avg,df_FMED_unplanned_avg,df_FNWE_unplanned_avg])
    return df_unplanned_avg

def raw_data_IEA():
    #GETTING IEA TABLE
    country_list = country()['country'].tolist()
    IEA =pd.DataFrame({'country':country_list,'iea_code':IEA_Countries})
    for i,n in zip(range(0,months_ahead), range(1,months_ahead + 1)):
        IEA = pd.merge(IEA, iea(0,i,n),how="inner", on='iea_code')
    IEA =IEA.drop(columns=['iea_code'])
    return IEA

def raw_data_JODI():
    #GETTING JODI TABLE
    country_list = country()['country'].tolist()
    JODI =pd.DataFrame({'country':country_list,'jodi_code':JODI_Countries})
    for i in zip(range(0,months_ahead)):
        JODI = pd.merge(JODI, jodi(0,i),how="inner", on='jodi_code')
    JODI =JODI.drop(columns=['jodi_code'])
    print(JODI)
    return JODI


def correlation_data_cor_margin_table():
    # GETTING MARGIN TABLE
    cor_region = ['nwecomplex', 'medcomplex']
    cor_nwe_margin = pd.DataFrame({'region': [cor_region[0]]})
    for i in zip(range(0, correlation_months)):
        cor_nwe_margin = pd.merge(cor_nwe_margin, margin(1, i, cor_region[0]), how="inner", on='region')
    # print(cor_nwe_margin)

    cor_med_margin = pd.DataFrame({'region': [cor_region[1]]})
    for i in zip(range(0, correlation_months)):
        cor_med_margin = pd.merge(cor_med_margin, margin(1, i, cor_region[1]), how="inner", on='region')
    # print(cor_med_margin)

    cor_margin_table = pd.concat([cor_nwe_margin, cor_med_margin])
    return cor_margin_table

def correlation_data_cor_nameplate_cap():
    #GETTING NAMEPLATE CAPACITY TABLE
    cor_nameplate_cap =country()
    for i in zip(range(0,correlation_months)):
        cor_nameplate_cap = pd.merge(cor_nameplate_cap, nameplate(1,i), how="inner", on='country')

    cor_FMED_nameplate_cap =pd.DataFrame({'country':['France Med']})
    for i in zip(range(0,correlation_months)):
        cor_FMED_nameplate_cap = pd.concat([cor_FMED_nameplate_cap, FMED_nameplate(1,i)],axis=1, join="inner")

    cor_FNWE_nameplate_cap =pd.DataFrame({'country':['France NWE']})
    for i in zip(range(0,correlation_months)):
        cor_FNWE_nameplate_cap = pd.concat([cor_FNWE_nameplate_cap, FNWE_nameplate(1,i)],axis=1, join="inner")

    cor_nameplate_cap= pd.concat([cor_nameplate_cap,cor_FMED_nameplate_cap,cor_FNWE_nameplate_cap])
    #print(FMED_p_k(10,11))
    #sys.exit()
    return cor_nameplate_cap

def correlation_data_cor_planned_known():
    #GETTING PLANNED KNOWN MAINTENANCE TABLE
    df_cor_planned_known =country()
    for i,n in zip(range(0,correlation_months), range(1,correlation_months + 1)):
        df_cor_planned_known = pd.merge(df_cor_planned_known, p_k(1,i,n), how="inner", on='country')

    df_cor_FMED_planned_known =pd.DataFrame({'country':['France Med']})
    for i,n in zip(range(0,correlation_months), range(1,correlation_months + 1)):
        df_cor_FMED_planned_known = pd.concat([df_cor_FMED_planned_known, FMED_p_k(1,i,n)],axis=1, join="inner")

    df_cor_FNWE_planned_known =pd.DataFrame({'country':['France NWE']})
    for i,n in zip(range(0,correlation_months), range(1,correlation_months + 1)):
        df_cor_FNWE_planned_known = pd.concat([df_cor_FNWE_planned_known, FNWE_p_k(1,i,n)],axis=1, join="inner")

    df_cor_planned_known= pd.concat([df_cor_planned_known,df_cor_FMED_planned_known,df_cor_FNWE_planned_known])
    return df_cor_planned_known

def correlation_data_cor_unplanned_known():
    #GETTING UNPLANNED KNOWN MAINTENANCE TABLE
    df_cor_unplanned_known =country()
    for i,n in zip(range(0,correlation_months), range(1,correlation_months + 1)):
        df_cor_unplanned_known = pd.merge(df_cor_unplanned_known, u_k(1,i,n), how="inner", on='country')

    df_cor_FMED_unplanned_known =pd.DataFrame({'country':['France Med']})
    for i,n in zip(range(0,correlation_months), range(1,correlation_months + 1)):
        df_cor_FMED_unplanned_known = pd.concat([df_cor_FMED_unplanned_known, FMED_u_k(1,i,n)],axis=1, join="inner")

    df_cor_FNWE_unplanned_known =pd.DataFrame({'country':['France NWE']})
    for i,n in zip(range(0,correlation_months), range(1,correlation_months + 1)):
        df_cor_FNWE_unplanned_known = pd.concat([df_cor_FNWE_unplanned_known, FNWE_u_k(1,i,n)],axis=1, join="inner")

    df_cor_unplanned_known= pd.concat([df_cor_unplanned_known,df_cor_FMED_unplanned_known,df_cor_FNWE_unplanned_known])
    return df_cor_unplanned_known

def correlation_data_cor_JODI():
    #GETTING JODI TABLE
    country_list = country()['country'].tolist()
    cor_JODI =pd.DataFrame({'country':country_list,'jodi_code':JODI_Countries})
    for i in zip(range(0,correlation_months)):
        cor_JODI = pd.merge(cor_JODI, jodi(1,i),how="inner", on='jodi_code')
    cor_JODI =cor_JODI.drop(columns=['jodi_code'])
    return cor_JODI

def get_max_range():
    max_run_range =pd.DataFrame({'index':['Max Range']})
    for i in zip(range(0, 12)):
        max_run_range = pd.concat([max_run_range, max_5yr_range(2,i)],axis=1, join="inner")
#     print(max_run_range)
    return max_run_range

def get_min_range():
    min_run_range =pd.DataFrame({'index':['Min Range']})
    for i in zip(range(0, 12)):
        min_run_range = pd.concat([min_run_range, min_5yr_range(2,i)],axis=1, join="inner")
#     print(min_run_range)
    return min_run_range

def get_planned_outage_forecast():
    # get forecast planned table
    df_planned_forecast = (df_planned_known.loc[:, df_planned_known.columns <= (datetime.now()+relativedelta(months=1)).strftime('%Y-%m-01')]).copy()
    ## for second and third month use 25% of avg as placeholder
    df_planned_known_placeholder_prompt_months = df_planned_known.loc[:,(df_planned_known.columns >= (datetime.now()+relativedelta(months=2)).strftime('%Y-%m-01'))&(df_planned_known.columns <= (datetime.now()+relativedelta(months=3)).strftime('%Y-%m-01'))]
    df_planned_avg_placeholder_prompt_months = ((df_planned_avg.loc[:, (df_planned_known.columns >= (datetime.now()+relativedelta(months=2)).strftime('%Y-%m-01'))&(df_planned_known.columns <= (datetime.now()+relativedelta(months=3)).strftime('%Y-%m-01'))]).copy())*0.25
    for header in df_planned_known_placeholder_prompt_months.columns:
        df_planned_forecast.loc[:,header]= np.where(df_planned_known_placeholder_prompt_months.loc[:,header] >= df_planned_avg_placeholder_prompt_months.loc[:,header], df_planned_known_placeholder_prompt_months[header], df_planned_avg_placeholder_prompt_months[header])
    ## for months 3+ use full 100% of avg
    df_planned_known_placeholder_back_months = df_planned_known.loc[:,df_planned_known.columns >= (datetime.now() + relativedelta(months=4)).strftime('%Y-%m-01')]
    df_planned_avg_placeholder_back_months = ((df_planned_avg.loc[:, df_planned_known.columns >= (datetime.now() + relativedelta(months=4)).strftime('%Y-%m-01')]).copy())
    for header in df_planned_known_placeholder_back_months.columns:
        df_planned_forecast.loc[:, header] = np.where(df_planned_known_placeholder_back_months.loc[:, header] >= df_planned_avg_placeholder_back_months.loc[:,header],df_planned_known_placeholder_back_months[header], df_planned_avg_placeholder_back_months[header])
    df_planned_forecast.loc['France'] = df_planned_forecast.loc['France Med'] + df_planned_forecast.loc['France NWE']
    return df_planned_forecast

def get_planned_outage_forecast_which_are_known():
    # get forecast planned table
    df_planned_forecast_which_are_known = (df_planned_known.loc[:, df_planned_known.columns <= (datetime.now()+relativedelta(months=1)).strftime('%Y-%m-01')]).copy()
    ## for second and third month use 25% of avg as placeholder
    df_planned_known_placeholder_prompt_months = df_planned_known.loc[:,(df_planned_known.columns >= (datetime.now()+relativedelta(months=2)).strftime('%Y-%m-01'))&(df_planned_known.columns <= (datetime.now()+relativedelta(months=3)).strftime('%Y-%m-01'))]
    df_planned_avg_placeholder_prompt_months = ((df_planned_avg.loc[:,(df_planned_known.columns >= (datetime.now()+relativedelta(months=2)).strftime('%Y-%m-01'))&(df_planned_known.columns <= (datetime.now()+relativedelta(months=3)).strftime('%Y-%m-01'))]).copy())*0.25
    for header in df_planned_known_placeholder_prompt_months.columns:
        df_planned_forecast_which_are_known.loc[:,header]= np.where(df_planned_known_placeholder_prompt_months.loc[:,header] >= df_planned_avg_placeholder_prompt_months.loc[:,header], df_planned_known_placeholder_prompt_months[header] , 0)
    ## for months 3+ use full 100% of avg
    df_planned_known_placeholder_back_months = df_planned_known.loc[:,df_planned_known.columns >= (datetime.now() + relativedelta(months=4)).strftime('%Y-%m-01')]
    df_planned_avg_placeholder_back_months = ((df_planned_avg.loc[:, df_planned_known.columns >= (datetime.now() + relativedelta(months=4)).strftime('%Y-%m-01')]).copy())
    for header in df_planned_known_placeholder_back_months.columns:
        df_planned_forecast_which_are_known.loc[:, header] = np.where(df_planned_known_placeholder_back_months.loc[:, header] >= df_planned_avg_placeholder_back_months.loc[:,header],df_planned_known_placeholder_back_months[header], 0)
    df_planned_forecast_which_are_known.loc['France'] = df_planned_forecast_which_are_known.loc['France Med'] + df_planned_forecast_which_are_known.loc['France NWE']
    return df_planned_forecast_which_are_known

def get_planned_outage_forecast_which_are_avg():
    df_planned_forecast_which_are_avg= df_planned_forecast.sub(df_planned_forecast_which_are_known,axis=0)
    return df_planned_forecast_which_are_avg


def get_unplanned_outage_forecast():
    # get forecast unplanned table
    ## get known values first
    df_unplanned_forecast = (df_unplanned_known.loc[:,df_unplanned_known.columns <= (datetime.now()).strftime('%Y-%m-01')]).copy()
    ## for forecast of future months get larger of known or 100% avg
    df_unplanned_known_placeholder = df_unplanned_known.loc[:, df_unplanned_known.columns > (datetime.now()).strftime('%Y-%m-01')]
    df_unplanned_avg_placeholder= df_unplanned_avg.loc[:, df_unplanned_known.columns > (datetime.now()).strftime('%Y-%m-01')]
    for header in df_unplanned_known_placeholder.columns:
        df_unplanned_forecast.loc[:,header]= np.where(df_unplanned_known_placeholder.loc[:,header] >= df_unplanned_avg_placeholder.loc[:,header], df_unplanned_known_placeholder[header] , df_unplanned_avg_placeholder[header])
        df_unplanned_forecast.loc['France'] = df_unplanned_forecast.loc['France Med'] + df_unplanned_forecast.loc['France NWE']
    return df_unplanned_forecast

def get_unplanned_outage_forecast_which_are_known():
    # get forecast unplanned table
    ## get known values first
    df_unplanned_forecast_which_are_known = (df_unplanned_known.loc[:,df_unplanned_known.columns <= (datetime.now()).strftime('%Y-%m-01')]).copy()
    ## for forecast of future months get larger of known or 100% avg
    df_unplanned_known_placeholder = df_unplanned_known.loc[:, df_unplanned_known.columns > (datetime.now()).strftime('%Y-%m-01')]
    df_unplanned_avg_placeholder= df_unplanned_avg.loc[:, df_unplanned_known.columns > (datetime.now()).strftime('%Y-%m-01')]
    for header in df_unplanned_known_placeholder.columns:
        df_unplanned_forecast_which_are_known.loc[:,header]= np.where(df_unplanned_known_placeholder.loc[:,header] >= df_unplanned_avg_placeholder.loc[:,header], df_unplanned_known_placeholder[header] , 0)
        df_unplanned_forecast_which_are_known.loc['France'] = df_unplanned_forecast_which_are_known.loc['France Med'] + df_unplanned_forecast_which_are_known.loc['France NWE']
    return df_unplanned_forecast_which_are_known

def get_unplanned_outage_forecast_which_are_avg():
    df_unplanned_forecast_which_are_avg = df_unplanned_forecast.sub(df_unplanned_forecast_which_are_known,axis=0)
    return df_unplanned_forecast_which_are_avg

def get_available_capacity():
    df_available_capacity = nameplate_cap.sub(df_planned_forecast,axis=0).sub(df_unplanned_forecast,axis=0)
    df_available_capacity[df_available_capacity < 0] = 0
    return df_available_capacity

def get_correlation():
    #change all external variables inline with correlation start date
    correct_jodi = (cor_JODI.loc[:, cor_JODI.columns >= (correlation_start_date).strftime('%Y-%m-01')]).copy()
    correct_nameplate_cap = (cor_nameplate_cap.loc[:, cor_nameplate_cap.columns >= (correlation_start_date).strftime('%Y-%m-01')]).copy()
    correct_planned_known = (df_cor_planned_known.loc[:, df_cor_planned_known.columns >= (correlation_start_date).strftime('%Y-%m-01')]).copy()
    correct_unplanned_known = (df_cor_unplanned_known.loc[:, df_cor_unplanned_known.columns >= (correlation_start_date).strftime('%Y-%m-01')]).copy()
    correct_margin_table = (cor_margin_table.loc[:, cor_margin_table.columns >= (correlation_start_date).strftime('%Y-%m-01')]).copy()
    print('Correlation Margins')
    print(correct_margin_table)
    # Calc Available Capacity
    cor_available_capacity = correct_nameplate_cap.sub(correct_planned_known,axis=0).sub(correct_unplanned_known,axis=0)
    cor_available_capacity[cor_available_capacity < 0] = 0
    #Add/Calc France NWE & France Med JODI
    correct_jodi.loc['France Med'] = correct_jodi.loc['France'] * ((cor_available_capacity.loc['France Med']) / (cor_available_capacity.loc['France']))
    correct_jodi.loc['France NWE'] = correct_jodi.loc['France'] * ((cor_available_capacity.loc['France NWE']) / (cor_available_capacity.loc['France']))
    #utilisation rate
    utilisation=correct_jodi.loc[:, correct_jodi.columns >= (correlation_start_date).strftime('%Y-%m-01')].div(cor_available_capacity,axis=0)
    utilisation.replace(np.inf, np.nan, inplace=True)
    adjusted_utilisation=pd.DataFrame(SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(utilisation.T).T)
    adjusted_utilisation.columns = utilisation.columns
    adjusted_utilisation.index = utilisation.index
    #utilisation = cor_JODI.div(df_available_capacity, axis=0)
    #getting correlation
    NWE_X=correct_margin_table.loc['nwecomplex']
    MED_X=correct_margin_table.loc['medcomplex']
    NWE_utilisation=adjusted_utilisation.loc[NWE_Countries]
    MED_utilisation=adjusted_utilisation.loc[Med_Countries]
    print('NWE Utilisation')
    print(NWE_utilisation)
    print('Med Utilisation')
    print(MED_utilisation)
    #where liner equation is y=mx+c
    correlation_params = pd.DataFrame(columns=['M','C'])
    for place in NWE_utilisation.index:
        Y = NWE_utilisation.loc[place]
        X = NWE_X
        anomaly = np.where(Y >= correlation_utilisation_upper_limit)
        anomaly = pd.Series(anomaly[0])
        Y = pd.Series(Y.reset_index(drop=True))
        X = pd.Series(X.reset_index(drop=True))
        if not anomaly.empty:
            #X = np.delete(X, anomaly)
            #Y = np.delete(Y, anomaly)
            #X = X.values.reshape(-1, 1)
            #Y = Y.values.reshape(-1, 1)
            #for position in anomaly:
            X = X.drop(anomaly)
            Y = Y.drop(anomaly)
            X = X.values.reshape(-1,1)
            Y = Y.values.reshape(-1,1)
        else:
            X = X.values.reshape(-1, 1)
            Y = Y.values.reshape(-1, 1)
        correlation_params.loc[place]=[(Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random').fit(X,Y)).coef_,(Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random').fit(X,Y)).intercept_]
    for place in MED_utilisation.index:
        Y = MED_utilisation.loc[place]
        X = MED_X
        anomaly = np.where(Y >= correlation_utilisation_upper_limit)
        anomaly = pd.Series(anomaly[0])
        Y = pd.Series(Y.reset_index(drop=True))
        X = pd.Series(X.reset_index(drop=True))
        if not anomaly.empty:
            #X = np.delete(X,anomaly)
            #Y = np.delete(Y,anomaly)
            #X = X.values.reshape(-1, 1)
            #Y = Y.values.reshape(-1, 1)
            #for position in anomaly:
            X = X.drop(anomaly)
            Y = Y.drop(anomaly)
            X = X.values.reshape(-1, 1)
            Y = Y.values.reshape(-1, 1)
        else:
            X = X.values.reshape(-1, 1)
            Y = Y.values.reshape(-1, 1)
        correlation_params.loc[place]=[(Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random').fit(X,Y)).coef_,(Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random').fit(X,Y)).intercept_]
    return correlation_params

def get_utilisation_forecast():
    utilisation_forecast = pd.DataFrame(data=None, columns=df_planned_known.columns, index=df_planned_known.index)
    NWE_utilisation_forecast=utilisation_forecast.loc[NWE_Countries]
    MED_utilisation_forecast=utilisation_forecast.loc[Med_Countries]
    NWE_X=margin_table.loc['nwecomplex']
    MED_X=margin_table.loc['medcomplex']
    for place in NWE_utilisation_forecast.index:
        utilisation_forecast.loc[place]=((correlation_params.loc[place]['M'][0])*NWE_X)+(correlation_params.loc[place]['C'][0])
    for place in MED_utilisation_forecast.index:
        utilisation_forecast.loc[place]=((correlation_params.loc[place]['M'][0])*MED_X)+(correlation_params.loc[place]['C'][0])
    return utilisation_forecast

def get_run_forecast():
    run_forecast = df_available_capacity.mul(utilisation_forecast,axis=0)
    run_forecast.loc['France'] = run_forecast.loc['France Med'] + run_forecast.loc['France NWE']
    return run_forecast

def get_regional_run_plot():
    # REGIONAL PLOT
    OECD_Europe_JODI = pd.DataFrame(data=None, columns=JODI_plot.columns)
    OECD_Europe_JODI.loc['OECD Europe'] = JODI_plot.sum(axis=0)
    OECD_Europe_total_forecast = pd.DataFrame(data=None, columns=df_planned_known.columns)
    OECD_Europe_total_forecast.loc['OECD Europe Forecast'] = run_forecast.sum(axis=0)
    last_week_forecast_total_forecast = pd.DataFrame(data=None, columns=last_week_forecast.columns)
    last_week_forecast_total_forecast.loc['Last Week OECD Europe Forecast'] = last_week_forecast.sum(axis=0)
    # sort into year [year before last,last year,this year]
    OECD_Europe_JODI_plot = [(datetime.now() - relativedelta(years=2)).strftime('%Y'),
                             (datetime.now() - relativedelta(years=1)).strftime('%Y'), (datetime.now()).strftime('%Y')]
    OECD_Europe_JODI_plot[0] = (OECD_Europe_JODI.loc[:,
                                (pd.to_datetime(OECD_Europe_JODI.columns)).strftime('%Y') == OECD_Europe_JODI_plot[
                                    0]]).copy()
    #OECD_Europe_JODI_plot[0] = OECD_Europe_JODI_plot[0].rename(index={'OECD Europe': (datetime.now() - relativedelta(years=2)).strftime('%Y') + "JODI Actual Run"})
    OECD_Europe_JODI_plot[0].replace(0, np.nan, inplace=True)

    OECD_Europe_JODI_plot[1] = (OECD_Europe_JODI.loc[:,
                                (pd.to_datetime(OECD_Europe_JODI.columns)).strftime('%Y') == OECD_Europe_JODI_plot[
                                    1]]).copy()
    #OECD_Europe_JODI_plot[1] = OECD_Europe_JODI_plot[1].rename( index={'OECD Europe': (datetime.now() - relativedelta(years=1)).strftime('%Y') + "JODI Actual Run"})
    OECD_Europe_JODI_plot[1].replace(0, np.nan, inplace=True)
    OECD_Europe_JODI_plot[2] = (OECD_Europe_JODI.loc[:,
                                (pd.to_datetime(OECD_Europe_JODI.columns)).strftime('%Y') == OECD_Europe_JODI_plot[
                                    2]]).copy()
    #OECD_Europe_JODI_plot[2] = OECD_Europe_JODI_plot[2].rename(index={'OECD Europe': datetime.now().strftime('%Y') + "JODI Actual Run"})
    OECD_Europe_JODI_plot[2].replace(0, np.nan, inplace=True)

    # prepare forecast for plot by limiting date to this year
    OECD_Europe_total_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                       (datetime.now()).strftime('%Y'),(datetime.now() + relativedelta(years=1)).strftime('%Y')]
    OECD_Europe_total_forecast_plot[0] = (OECD_Europe_total_forecast.loc[:,
                                          (pd.to_datetime(OECD_Europe_total_forecast.columns)).strftime('%Y') ==
                                          OECD_Europe_total_forecast_plot[0]]).copy()
    #OECD_Europe_total_forecast_plot[0] = OECD_Europe_total_forecast_plot[0].rename( index={'OECD Europe Forecast': (datetime.now() - relativedelta(years=1)).strftime('%Y') + "Forecast Run"})
    OECD_Europe_total_forecast_plot[1] = (OECD_Europe_total_forecast.loc[:,
                                          (pd.to_datetime(OECD_Europe_total_forecast.columns)).strftime('%Y') ==
                                          OECD_Europe_total_forecast_plot[1]]).copy()
    #OECD_Europe_total_forecast_plot[1] = OECD_Europe_total_forecast_plot[1].rename(index={'OECD Europe Forecast': datetime.now().strftime('%Y') + "Forecast Run"})
    OECD_Europe_total_forecast_plot[2] = (OECD_Europe_total_forecast.loc[:,
                                          (pd.to_datetime(OECD_Europe_total_forecast.columns)).strftime('%Y') ==
                                          OECD_Europe_total_forecast_plot[2]]).copy()
    # OECD_Europe_total_forecast_plot[2] = OECD_Europe_total_forecast_plot[2].rename(index={'OECD Europe Forecast': datetime.now().strftime('%Y') + "Forecast Run"})
    last_week_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                       (datetime.now()).strftime('%Y'),(datetime.now() + relativedelta(years=1)).strftime('%Y')]

    last_week_forecast_plot[0] = (last_week_forecast_total_forecast.loc[:,(pd.to_datetime(last_week_forecast_total_forecast.columns)).strftime('%Y') ==last_week_forecast_plot[0]]).copy()
    last_week_forecast_plot[1] = (last_week_forecast_total_forecast.loc[:,(pd.to_datetime(last_week_forecast_total_forecast.columns)).strftime('%Y') ==last_week_forecast_plot[1]]).copy()
    last_week_forecast_plot[2] = (last_week_forecast_total_forecast.loc[:,(pd.to_datetime(last_week_forecast_total_forecast.columns)).strftime('%Y') ==last_week_forecast_plot[2]]).copy()


    # change column headers back to datetime in day of year format to allow all line on same graph
    OECD_Europe_total_forecast_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_total_forecast_plot[0].columns)).dayofyear, format='%j')
    OECD_Europe_total_forecast_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_total_forecast_plot[1].columns)).dayofyear, format='%j')
    OECD_Europe_total_forecast_plot[2].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_total_forecast_plot[2].columns)).dayofyear, format='%j')
    OECD_Europe_JODI_plot[0].columns = pd.to_datetime((pd.to_datetime(OECD_Europe_JODI_plot[0].columns)).dayofyear,
                                                      format='%j')
    OECD_Europe_JODI_plot[1].columns = pd.to_datetime((pd.to_datetime(OECD_Europe_JODI_plot[1].columns)).dayofyear,
                                                      format='%j')
    OECD_Europe_JODI_plot[2].columns = pd.to_datetime((pd.to_datetime(OECD_Europe_JODI_plot[2].columns)).dayofyear,
                                                      format='%j')
    max_range.columns = pd.to_datetime((pd.to_datetime(max_range.columns)).dayofyear,
                               format='%j')

    min_range.columns =pd.to_datetime((pd.to_datetime(min_range.columns)).dayofyear,
                                                      format='%j')

    last_week_forecast_plot[0].columns = pd.to_datetime((pd.to_datetime(last_week_forecast_plot[0].columns)).dayofyear, format='%j')
    last_week_forecast_plot[1].columns = pd.to_datetime((pd.to_datetime(last_week_forecast_plot[1].columns)).dayofyear, format='%j')
    last_week_forecast_plot[2].columns = pd.to_datetime((pd.to_datetime(last_week_forecast_plot[2].columns)).dayofyear, format='%j')

    # Plot Regional Graph
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(1, figsize=(15, 7.5))

    # sns.set_style('seaborn-dark-palette')
    ax.plot(last_week_forecast_plot[0].columns, last_week_forecast_plot[0].loc['Last Week OECD Europe Forecast'],
            marker='.', linestyle='dashed', color='grey', markerfacecolor='grey', label='_nolegend_')
    ax.plot(last_week_forecast_plot[1].columns, last_week_forecast_plot[1].loc['Last Week OECD Europe Forecast'],
            marker='.', linestyle='dashed', color='grey', markerfacecolor='grey', label='Last Week Forecast')
    #ax.plot(last_week_forecast_plot[2].columns, last_week_forecast_plot[2].loc['Last Week OECD Europe Forecast'],
    #        marker='.', linestyle='dashed', color='grey', markerfacecolor='grey', label='_nolegend_')
    ax.plot(OECD_Europe_JODI_plot[0].columns, OECD_Europe_JODI_plot[0].loc['OECD Europe'], marker='.',color='green',markerfacecolor='green', label=(datetime.now() - relativedelta(years=2)).strftime('%Y') + " JODI Actual Run")
    ax.plot(OECD_Europe_JODI_plot[1].columns, OECD_Europe_JODI_plot[1].loc['OECD Europe'], marker='.',color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " JODI Actual Run")
    ax.plot(OECD_Europe_JODI_plot[2].columns, OECD_Europe_JODI_plot[2].loc['OECD Europe'], marker='.',color='red', markerfacecolor='red',label=datetime.now().strftime('%Y') + " JODI Actual Run")
    ax.plot(OECD_Europe_total_forecast_plot[0].columns, OECD_Europe_total_forecast_plot[0].loc['OECD Europe Forecast'], marker='.', linestyle='dashed',color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Forecast Run")
    ax.plot(OECD_Europe_total_forecast_plot[1].columns, OECD_Europe_total_forecast_plot[1].loc['OECD Europe Forecast'], marker='.',linestyle='dashed',color='red',markerfacecolor='red',label= datetime.now().strftime('%Y') + " Forecast Run")
#     ax.plot(OECD_Europe_total_forecast_plot[2].columns, OECD_Europe_total_forecast_plot[2].loc['OECD Europe Forecast'], marker='.',linestyle='dashed',color='sienna',markerfacecolor='sienna',label= (datetime.now() + relativedelta(years=1)).strftime('%Y') + " Forecast Run")

    plt.fill_between(min_range.columns,min_range.loc['Min Range'],max_range.loc['Max Range'],color='grey',alpha=0.2, label="5yr Range")
    ax.set(ylabel="Run Rate (kb/d)")
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax.yaxis.set_major_formatter(tick)
    major_format = mdates.DateFormatter('%b')
    ax.xaxis.set_major_formatter(major_format)
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.title('OECD Europe Runs Forecast')
    ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=4)
    fig.subplots_adjust(bottom=0.6)  # <-- Change the 0.02 to work for your plot.
    fig.tight_layout()
    fig.savefig(img_subj,bbox_inches='tight')
    plt.close()

def get_country_run_plots():
    # Country PLOT
    # sort into year [year before last,last year,this year]
    NWE_country_JODI_plot = [(datetime.now() - relativedelta(years=2)).strftime('%Y'),
                             (datetime.now() - relativedelta(years=1)).strftime('%Y'), (datetime.now()).strftime('%Y')]
    NWE_country_JODI_plot[0] = (JODI_plot.loc[
        NWE_Countries, (pd.to_datetime(JODI_plot.columns)).strftime('%Y') == NWE_country_JODI_plot[0]]).copy()
    NWE_country_JODI_plot[0].replace(0, np.nan, inplace=True)
    NWE_country_JODI_plot[1] = (JODI_plot.loc[
        NWE_Countries, (pd.to_datetime(JODI_plot.columns)).strftime('%Y') == NWE_country_JODI_plot[1]]).copy()
    NWE_country_JODI_plot[1].replace(0, np.nan, inplace=True)
    NWE_country_JODI_plot[2] = (JODI_plot.loc[
        NWE_Countries, (pd.to_datetime(JODI_plot.columns)).strftime('%Y') == NWE_country_JODI_plot[2]]).copy()
    NWE_country_JODI_plot[2].replace(0, np.nan, inplace=True)

    MED_country_JODI_plot = [(datetime.now() - relativedelta(years=2)).strftime('%Y'),
                             (datetime.now() - relativedelta(years=1)).strftime('%Y'), (datetime.now()).strftime('%Y')]
    MED_country_JODI_plot[0] = (JODI_plot.loc[
        Med_Countries, (pd.to_datetime(JODI_plot.columns)).strftime('%Y') ==
        MED_country_JODI_plot[0]]).copy()
    MED_country_JODI_plot[0].replace(0, np.nan, inplace=True)
    MED_country_JODI_plot[1] = (JODI_plot.loc[
        Med_Countries, (pd.to_datetime(JODI_plot.columns)).strftime('%Y') ==
        MED_country_JODI_plot[1]]).copy()
    MED_country_JODI_plot[1].replace(0, np.nan, inplace=True)
    MED_country_JODI_plot[2] = (JODI_plot.loc[
        Med_Countries, (pd.to_datetime(JODI_plot.columns)).strftime('%Y') ==
        MED_country_JODI_plot[2]]).copy()
    MED_country_JODI_plot[2].replace(0, np.nan, inplace=True)

    # prepare forecast for plot by limiting date to this year
    NWE_country_total_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                       (datetime.now()).strftime('%Y')]
    NWE_country_total_forecast_plot[0] = (run_forecast.loc[
        NWE_Countries, (pd.to_datetime(run_forecast.columns)).strftime('%Y') == NWE_country_total_forecast_plot[
            0]]).copy()
    NWE_country_total_forecast_plot[1] = (run_forecast.loc[
        NWE_Countries, (pd.to_datetime(run_forecast.columns)).strftime('%Y') == NWE_country_total_forecast_plot[
            1]]).copy()

    MED_country_total_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                       (datetime.now()).strftime('%Y')]
    MED_country_total_forecast_plot[0] = (run_forecast.loc[
        Med_Countries, (pd.to_datetime(run_forecast.columns)).strftime('%Y') ==
        MED_country_total_forecast_plot[0]]).copy()
    MED_country_total_forecast_plot[1] = (run_forecast.loc[
        Med_Countries, (pd.to_datetime(run_forecast.columns)).strftime('%Y') ==
        MED_country_total_forecast_plot[1]]).copy()

    # change column headers back to datetime in day of year format to allow all line on same graph
    NWE_country_total_forecast_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_total_forecast_plot[0].columns)).dayofyear, format='%j')
    NWE_country_total_forecast_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_total_forecast_plot[1].columns)).dayofyear, format='%j')
    NWE_country_JODI_plot[0].columns = pd.to_datetime((pd.to_datetime(NWE_country_JODI_plot[0].columns)).dayofyear,
                                                      format='%j')
    NWE_country_JODI_plot[1].columns = pd.to_datetime((pd.to_datetime(NWE_country_JODI_plot[1].columns)).dayofyear,
                                                      format='%j')
    NWE_country_JODI_plot[2].columns = pd.to_datetime((pd.to_datetime(NWE_country_JODI_plot[2].columns)).dayofyear,
                                                      format='%j')

    MED_country_total_forecast_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_total_forecast_plot[0].columns)).dayofyear, format='%j')
    MED_country_total_forecast_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_total_forecast_plot[1].columns)).dayofyear, format='%j')
    MED_country_JODI_plot[0].columns = pd.to_datetime((pd.to_datetime(MED_country_JODI_plot[0].columns)).dayofyear,
                                                      format='%j')
    MED_country_JODI_plot[1].columns = pd.to_datetime((pd.to_datetime(MED_country_JODI_plot[1].columns)).dayofyear,
                                                      format='%j')
    MED_country_JODI_plot[2].columns = pd.to_datetime((pd.to_datetime(MED_country_JODI_plot[2].columns)).dayofyear,
                                                      format='%j')

    # Plot NWE Country Graph
    plt.style.use('seaborn-darkgrid')
    fig1, axes = plt.subplots(nrows=5, ncols=2, figsize=(22, 16), sharex=True, constrained_layout=False)
    # sns.set_style('seaborn-dark-palette')
    for i, ax1 in enumerate(axes.flatten()):
        #ax1.plot(NWE_country_JODI_plot[0].columns, NWE_country_JODI_plot[0].iloc[i],marker='.',color='green',markerfacecolor='green',label=(datetime.now() - relativedelta(years=2)).strftime('%Y') + " JODI Actual Run")
        ax1.plot(NWE_country_JODI_plot[1].columns, NWE_country_JODI_plot[1].iloc[i], marker='.',color='violet',markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " JODI Actual Run")
        ax1.plot(NWE_country_JODI_plot[2].columns, NWE_country_JODI_plot[2].iloc[i],marker='.',color='red',markerfacecolor='red',label=datetime.now().strftime('%Y') + " JODI Actual Run")
        ax1.plot(NWE_country_total_forecast_plot[0].columns, NWE_country_total_forecast_plot[0].iloc[i], marker='.',linestyle='dashed',color='violet',markerfacecolor='violet', label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Forecast Run")
        ax1.plot(NWE_country_total_forecast_plot[1].columns, NWE_country_total_forecast_plot[1].iloc[i],marker='.',linestyle='dashed',color='red',markerfacecolor='red',label=datetime.now().strftime('%Y') + " Forecast Run")

        ax1.set_title(NWE_country_JODI_plot[0].T.columns[i])
        fmt = '{x:,.0f}'
        tick = mtick.StrMethodFormatter(fmt)
        ax1.yaxis.set_major_formatter(tick)
        major_format = mdates.DateFormatter('%b')
        ax1.xaxis.set_major_formatter(major_format)
        ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        plt.tight_layout()
    handles, labels = ax1.get_legend_handles_labels()
    plt.setp(axes[:, 0],ylabel="Run Rate (kb/d)")
    fig1.legend(handles, labels,loc='lower center', bbox_to_anchor=(0.5, 0), fancybox=True, shadow=True, ncol=4)
    fig1.subplots_adjust(bottom=0.8)  # <-- Change the 0.02 to work for your plot.
    fig1.tight_layout()
    fig1.savefig(img_subj1,bbox_inches='tight')
    plt.close()

    # Plot MED Country Graph
    plt.style.use('seaborn-darkgrid')
    fig2, axes = plt.subplots(nrows=6, ncols=2, figsize=(22, 16), sharex=True, constrained_layout=False)
    # sns.set_style('seaborn-dark-palette')
    for i, ax2 in enumerate(axes.flatten()):
        # ax2.plot(MED_country_JODI_plot[0].columns, MED_country_JODI_plot[0].iloc[i],marker='.',color='green',markerfacecolor='green',label=(datetime.now() - relativedelta(years=2)).strftime('%Y') + " JODI Actual Run")
        ax2.plot(MED_country_JODI_plot[1].columns, MED_country_JODI_plot[1].iloc[i], marker='.', color='violet',markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " JODI Actual Run")
        ax2.plot(MED_country_JODI_plot[2].columns, MED_country_JODI_plot[2].iloc[i], marker='.', color='red', markerfacecolor='red', label=datetime.now().strftime('%Y') + " JODI Actual Run")
        ax2.plot(MED_country_total_forecast_plot[0].columns, MED_country_total_forecast_plot[0].iloc[i], marker='.',linestyle='dashed', color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Forecast Run")
        ax2.plot(MED_country_total_forecast_plot[1].columns, MED_country_total_forecast_plot[1].iloc[i], marker='.',linestyle='dashed', color='red', markerfacecolor='red',label=datetime.now().strftime('%Y') + " Forecast Run")


        ax2.set_title(MED_country_JODI_plot[0].T.columns[i])
        fmt = '{x:,.0f}'
        tick = mtick.StrMethodFormatter(fmt)
        ax2.yaxis.set_major_formatter(tick)
        major_format = mdates.DateFormatter('%b')
        ax2.xaxis.set_major_formatter(major_format)
        ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        plt.tight_layout()

    handles, labels = ax2.get_legend_handles_labels()
    plt.setp(axes[:, 0],ylabel="Run Rate (kb/d)")
    fig2.legend(handles, labels,loc='lower center', bbox_to_anchor=(0.5, 0), fancybox=True, shadow=True, ncol=4)
    fig2.subplots_adjust(bottom=0.8)  # <-- Change the 0.02 to work for your plot.
    fig2.tight_layout()
    fig2.savefig(img_subj2,bbox_inches='tight')
    plt.close()


#Plotting Maintenance Graphs
def get_maintenance_plot():
    # Get Total
    planned_forecast_which_are_known_total = pd.DataFrame(data=None,
                                                          columns=planned_forecast_which_are_known_join.columns)
    unplanned_forecast_which_are_known_total = pd.DataFrame(data=None,
                                                            columns=planned_forecast_which_are_known_join.columns)
    planned_forecast_which_are_avg_total = pd.DataFrame(data=None, columns=planned_forecast_which_are_known_join.columns)
    unplanned_forecast_which_are_avg_total = pd.DataFrame(data=None, columns=planned_forecast_which_are_known_join.columns)

    planned_forecast_which_are_known_total.loc['Planned TAR Known'] = (
        planned_forecast_which_are_known_join.drop(['France'])).sum(
        axis=0)
    unplanned_forecast_which_are_known_total.loc['Unplanned Outages Known'] = (
        unplanned_forecast_which_are_known_join.drop(['France'])).sum(axis=0)
    planned_forecast_which_are_avg_total.loc['Planned TAR Placeholder'] = (
        df_planned_forecast_which_are_avg.drop(['France'])).sum(axis=0)
    unplanned_forecast_which_are_avg_total.loc['Unplanned Outages Placeholder'] = (
        df_unplanned_forecast_which_are_avg.drop(['France'])).sum(axis=0)
    total_maintenance = pd.DataFrame(data=None, columns=planned_forecast_which_are_known_total.columns)
    total_maintenance.loc['Historic'] = planned_forecast_which_are_known_total.loc['Planned TAR Known'] + unplanned_forecast_which_are_known_total.loc['Unplanned Outages Known']

    planned_forecast_which_are_known_total = planned_forecast_which_are_known_total.loc[:,
                                             (pd.to_datetime(planned_forecast_which_are_known_total.columns)).strftime(
                                                 '%Y') == datetime.now().strftime('%Y')]
    unplanned_forecast_which_are_known_total = unplanned_forecast_which_are_known_total.loc[:, (pd.to_datetime(
        unplanned_forecast_which_are_known_total.columns)).strftime('%Y') == datetime.now().strftime('%Y')]
    planned_forecast_which_are_avg_total = planned_forecast_which_are_avg_total.loc[:,
                                           (pd.to_datetime(planned_forecast_which_are_avg_total.columns)).strftime(
                                               '%Y') == datetime.now().strftime('%Y')]
    unplanned_forecast_which_are_avg_total = unplanned_forecast_which_are_avg_total.loc[:,
                                             (pd.to_datetime(unplanned_forecast_which_are_avg_total.columns)).strftime(
                                                 '%Y') == datetime.now().strftime('%Y')]

    total_maintenance_last_year = total_maintenance.loc[:,
                                             (pd.to_datetime(total_maintenance.columns)).strftime(
                                                 '%Y') == (datetime.now() - relativedelta(years=1)).strftime('%Y')]
    total_maintenance_year_before_last = total_maintenance.loc[:,
                                  (pd.to_datetime(total_maintenance.columns)).strftime(
                                      '%Y') == (datetime.now() - relativedelta(years=2)).strftime('%Y')]

    planned_forecast_which_are_known_total.columns = pd.to_datetime(
        (pd.to_datetime(planned_forecast_which_are_known_total.columns)).dayofyear, format='%j')
    unplanned_forecast_which_are_known_total.columns = pd.to_datetime(
        (pd.to_datetime(unplanned_forecast_which_are_known_total.columns)).dayofyear, format='%j')
    planned_forecast_which_are_avg_total.columns = pd.to_datetime(
        (pd.to_datetime(planned_forecast_which_are_avg_total.columns)).dayofyear, format='%j')
    unplanned_forecast_which_are_avg_total.columns = pd.to_datetime(
        (pd.to_datetime(unplanned_forecast_which_are_avg_total.columns)).dayofyear, format='%j')

    total_maintenance_last_year.columns = pd.to_datetime(
        (pd.to_datetime(total_maintenance_last_year.columns)).dayofyear, format='%j')
    total_maintenance_year_before_last.columns = pd.to_datetime(
        (pd.to_datetime(total_maintenance_year_before_last.columns)).dayofyear, format='%j')
    plt.style.use('seaborn-darkgrid')
    fig3, ax3 = plt.subplots(1, figsize=(15, 8))
    barWidth = 5

    bottom0 = planned_forecast_which_are_known_total.loc['Planned TAR Known']
    bottom1 = np.add(bottom0, unplanned_forecast_which_are_known_total.loc['Unplanned Outages Known'])
    bottom2 = np.add(bottom1, planned_forecast_which_are_avg_total.loc['Planned TAR Placeholder'])

    plt.bar(planned_forecast_which_are_known_total.columns,
            planned_forecast_which_are_known_total.loc['Planned TAR Known'], color='grey', edgecolor='white',
            width=barWidth,label= datetime.now().strftime('%Y') + " Planned TAR (Known)")
    plt.bar(unplanned_forecast_which_are_known_total.columns,
            unplanned_forecast_which_are_known_total.loc['Unplanned Outages Known'], bottom=bottom0, color='orange',
            edgecolor='white', width=barWidth,label= datetime.now().strftime('%Y') + " Unplanned Outage (Known)")
    plt.bar(planned_forecast_which_are_avg_total.columns,
            planned_forecast_which_are_avg_total.loc['Planned TAR Placeholder'], bottom=bottom1, color='grey',
            edgecolor='white', width=barWidth, hatch="/",label= datetime.now().strftime('%Y') + " Planned TAR (Placeholder)")
    plt.bar(unplanned_forecast_which_are_avg_total.columns,
            unplanned_forecast_which_are_avg_total.loc['Unplanned Outages Placeholder'], bottom=bottom2,
            color='orange', edgecolor='white', width=barWidth, hatch="/",label= datetime.now().strftime('%Y') + " Unplanned Outage (Placeholder)")
    ax3.plot(total_maintenance_last_year.columns, total_maintenance_last_year.loc['Historic'], marker='.',color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Total Maintenance")
#     ax3.plot(total_maintenance_year_before_last.columns, total_maintenance_year_before_last.loc['Historic'],marker='.',color='green',markerfacecolor='green',label= (datetime.now() - relativedelta(years=2)).strftime('%Y') + " Total Maintenance")

    ax3.set(ylabel="Offline Capacity (kb/d)")
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    ax3.yaxis.set_major_formatter(tick)
    major_format = mdates.DateFormatter("%b")
    ax3.xaxis.set_major_formatter(major_format)
    ax3.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

    # x_ticks[0].label1.set_visible(False) ## set first x tick label invisible
    # x_ticks[-1].label1.set_visible(False) ## set last x tick label invisible

    plt.ylabel('Capacity Offline (kb/d)')
    plt.title('OECD Europe Maintenance Forecast')
    ax3.legend(loc='lower center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=3)
    fig3.subplots_adjust(bottom=0.6)  # <-- Change the 0.02 to work for your plot.
    fig3.tight_layout()
    fig3.savefig(img_subj3,bbox_inches='tight')
    plt.close()

def get_country_utilisation_plots():
    # Utilisation graphs
    available_capacity_join = (nameplate_capacity_join.sub(planned_forecast_which_are_known_join, axis=0).sub(
        unplanned_forecast_which_are_known_join, axis=0))
    available_capacity_join[available_capacity_join<0]=0
    utilisation_plot = JODI_join.div(available_capacity_join, axis=0)
    utilisation_plot = utilisation_plot.drop(['France'])
    utilisation_plot.replace(np.inf, np.nan, inplace=True)
    utilisation_plot.replace(0, np.nan, inplace=True)

    # Country PLOT
    # sort into year [year before last,last year,this year]
    NWE_country_utilisation_plot = [(datetime.now() - relativedelta(years=2)).strftime('%Y'),
                                    (datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                    (datetime.now()).strftime('%Y')]
    NWE_country_utilisation_plot[0] = (utilisation_plot.loc[
        NWE_Countries, (pd.to_datetime(utilisation_plot.columns)).strftime('%Y') == NWE_country_utilisation_plot[
            0]]).copy()
    NWE_country_utilisation_plot[0].replace(0, np.nan, inplace=True)
    NWE_country_utilisation_plot[1] = (utilisation_plot.loc[
        NWE_Countries, (pd.to_datetime(utilisation_plot.columns)).strftime('%Y') == NWE_country_utilisation_plot[
            1]]).copy()
    NWE_country_utilisation_plot[1].replace(0, np.nan, inplace=True)
    NWE_country_utilisation_plot[2] = (utilisation_plot.loc[
        NWE_Countries, (pd.to_datetime(utilisation_plot.columns)).strftime('%Y') == NWE_country_utilisation_plot[
            2]]).copy()
    NWE_country_utilisation_plot[2].replace(0, np.nan, inplace=True)

    MED_country_utilisation_plot = [(datetime.now() - relativedelta(years=2)).strftime('%Y'),
                                    (datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                    (datetime.now()).strftime('%Y')]
    MED_country_utilisation_plot[0] = (utilisation_plot.loc[
        Med_Countries, (pd.to_datetime(utilisation_plot.columns)).strftime('%Y') ==
        MED_country_utilisation_plot[0]]).copy()
    MED_country_utilisation_plot[0].replace(0, np.nan, inplace=True)
    MED_country_utilisation_plot[1] = (utilisation_plot.loc[
        Med_Countries, (pd.to_datetime(utilisation_plot.columns)).strftime('%Y') ==
        MED_country_utilisation_plot[1]]).copy()
    MED_country_utilisation_plot[1].replace(0, np.nan, inplace=True)
    MED_country_utilisation_plot[2] = (utilisation_plot.loc[
        Med_Countries, (pd.to_datetime(utilisation_plot.columns)).strftime('%Y') ==
        MED_country_utilisation_plot[2]]).copy()
    MED_country_utilisation_plot[2].replace(0, np.nan, inplace=True)

    # prepare forecast for plot by limiting date to this year
    NWE_country_utilisation_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                             (datetime.now()).strftime('%Y')]
    NWE_country_utilisation_forecast_plot[0] = (utilisation_forecast.loc[
        NWE_Countries, (pd.to_datetime(utilisation_forecast.columns)).strftime('%Y') ==
        NWE_country_utilisation_forecast_plot[0]]).copy()
    NWE_country_utilisation_forecast_plot[1] = (utilisation_forecast.loc[
        NWE_Countries, (pd.to_datetime(utilisation_forecast.columns)).strftime('%Y') ==
        NWE_country_utilisation_forecast_plot[1]]).copy()

    MED_country_utilisation_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                             (datetime.now()).strftime('%Y')]
    MED_country_utilisation_forecast_plot[0] = (utilisation_forecast.loc[
        Med_Countries, (pd.to_datetime(utilisation_forecast.columns)).strftime('%Y') ==
        MED_country_utilisation_forecast_plot[0]]).copy()
    MED_country_utilisation_forecast_plot[1] = (utilisation_forecast.loc[
        Med_Countries, (pd.to_datetime(utilisation_forecast.columns)).strftime('%Y') ==
        MED_country_utilisation_forecast_plot[1]]).copy()

    # change column headers back to datetime in day of year format to allow all line on same graph
    NWE_country_utilisation_forecast_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_utilisation_forecast_plot[0].columns)).dayofyear, format='%j')
    NWE_country_utilisation_forecast_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_utilisation_forecast_plot[1].columns)).dayofyear, format='%j')
    NWE_country_utilisation_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_utilisation_plot[0].columns)).dayofyear, format='%j')
    NWE_country_utilisation_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_utilisation_plot[1].columns)).dayofyear, format='%j')
    NWE_country_utilisation_plot[2].columns = pd.to_datetime(
        (pd.to_datetime(NWE_country_utilisation_plot[2].columns)).dayofyear, format='%j')

    MED_country_utilisation_forecast_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_utilisation_forecast_plot[0].columns)).dayofyear, format='%j')
    MED_country_utilisation_forecast_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_utilisation_forecast_plot[1].columns)).dayofyear, format='%j')
    MED_country_utilisation_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_utilisation_plot[0].columns)).dayofyear, format='%j')
    MED_country_utilisation_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_utilisation_plot[1].columns)).dayofyear, format='%j')
    MED_country_utilisation_plot[2].columns = pd.to_datetime(
        (pd.to_datetime(MED_country_utilisation_plot[2].columns)).dayofyear, format='%j')

    # Plot NWE Country Graph
    plt.style.use('seaborn-darkgrid')
    fig4, axes = plt.subplots(nrows=5, ncols=2, figsize=(22, 16), sharex=True, constrained_layout=False)
    # sns.set_style('seaborn-dark-palette')
    for i, ax4 in enumerate(axes.flatten()):
        # ax4.plot(NWE_country_utilisation_plot[0].columns, NWE_country_utilisation_plot[0].iloc[i],marker='.',color='green',markerfacecolor='green',label=(datetime.now() - relativedelta(years=2)).strftime('%Y') + " Actual Utilisation")
        ax4.plot(NWE_country_utilisation_plot[1].columns, NWE_country_utilisation_plot[1].iloc[i], marker='.', color='violet',markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Actual Utilisation")
        ax4.plot(NWE_country_utilisation_plot[2].columns, NWE_country_utilisation_plot[2].iloc[i], marker='.', color='red',markerfacecolor='red', label=datetime.now().strftime('%Y') + " Actual Utilisation")
        ax4.plot(NWE_country_utilisation_forecast_plot[0].columns, NWE_country_utilisation_forecast_plot[0].iloc[i], marker='.',linestyle='dashed', color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Forecast Utilisation")
        ax4.plot(NWE_country_utilisation_forecast_plot[1].columns, NWE_country_utilisation_forecast_plot[1].iloc[i], marker='.',linestyle='dashed', color='red', markerfacecolor='red',label=datetime.now().strftime('%Y') + " Forecast Utilisation")

        ax4.set_title(NWE_country_utilisation_plot[0].T.columns[i])
        major_format = mdates.DateFormatter('%b')
        ax4.xaxis.set_major_formatter(major_format)
        ax4.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        ax4.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))
        plt.tight_layout()
    handles, labels = ax4.get_legend_handles_labels()
    #fig4.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 0), fancybox=True, shadow=True, ncol=4)
    plt.setp(axes[:, 0],ylabel="Utilisation Rate")
    fig4.legend(handles, labels,loc='lower center', bbox_to_anchor=(0.5, 0), fancybox=True, shadow=True, ncol=4)
    fig4.subplots_adjust(bottom=0.8)  # <-- Change the 0.02 to work for your plot.
    fig4.tight_layout()
    fig4.savefig(img_subj4,bbox_inches='tight')
    plt.close()

    # Plot MED Country Graph
    plt.style.use('seaborn-darkgrid')
    fig5, axes = plt.subplots(nrows=6, ncols=2, figsize=(22, 16), sharex=True, constrained_layout=False)
    # sns.set_style('seaborn-dark-palette')
    for i, ax5 in enumerate(axes.flatten()):
        # ax5.plot(MED_country_utilisation_plot[0].columns, MED_country_utilisation_plot[0].iloc[i],marker='.',color='green',markerfacecolor='green',label=(datetime.now() - relativedelta(years=2)).strftime('%Y') + " Actual Utilisation")
        ax5.plot(MED_country_utilisation_plot[1].columns, MED_country_utilisation_plot[1].iloc[i], marker='.',color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Actual Utilisation")
        ax5.plot(MED_country_utilisation_plot[2].columns, MED_country_utilisation_plot[2].iloc[i], marker='.',color='red', markerfacecolor='red', label=datetime.now().strftime('%Y') + " Actual Utilisation")
        ax5.plot(MED_country_utilisation_forecast_plot[0].columns, MED_country_utilisation_forecast_plot[0].iloc[i],marker='.', linestyle='dashed', color='violet', markerfacecolor='violet', label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Forecast Utilisation")
        ax5.plot(MED_country_utilisation_forecast_plot[1].columns, MED_country_utilisation_forecast_plot[1].iloc[i],marker='.', linestyle='dashed', color='red', markerfacecolor='red',label=datetime.now().strftime('%Y') + " Forecast Utilisation")


        ax5.set_title(MED_country_utilisation_plot[0].T.columns[i])
        major_format = mdates.DateFormatter('%b')
        ax5.xaxis.set_major_formatter(major_format)
        ax5.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        ax5.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))
        plt.tight_layout()
    handles, labels = ax5.get_legend_handles_labels()
    plt.setp(axes[:, 0], ylabel="Utilisation Rate")
    fig5.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 0), fancybox=True, shadow=True, ncol=4)
    fig5.subplots_adjust(bottom=0.8)  # <-- Change the 0.02 to work for your plot.
    fig5.tight_layout()
    fig5.savefig(img_subj5,bbox_inches='tight')
    plt.close()

def get_regional_utilisation_plots():
    # Utilisation graphs
    OECD_Europe_utilisation = pd.DataFrame(data=None, columns=JODI_join.columns)
    OECD_Europe_available_cap = (((nameplate_capacity_join.drop(
            ['France'])).sum(
            axis=0)).sub(((planned_forecast_which_are_known_join.drop(
            ['France'])).sum(
            axis=0)), axis=0).sub(((unplanned_forecast_which_are_known_join.drop(
            ['France'])).sum(axis=0)),
                         axis=0))
    OECD_Europe_available_cap[OECD_Europe_available_cap < 0] = 0

    OECD_Europe_utilisation.loc['OECD Europe Utilisation'] = ((JODI_join.drop(
        ['France'])).sum(axis=0)).div(
        OECD_Europe_available_cap, axis=0)
    OECD_Europe_utilisation.replace(np.inf, 0, inplace=True)
    OECD_Europe_utilisation.replace(0, np.nan, inplace=True)
    OECD_Europe_utilisation_5yr_range = pd.melt(OECD_Europe_utilisation.reset_index(), id_vars='index',
                                                value_vars=OECD_Europe_utilisation.columns, var_name='date',
                                                value_name='utilisation')

    OECD_Europe_utilisation_5yr_range = OECD_Europe_utilisation_5yr_range.loc[(OECD_Europe_utilisation_5yr_range[
                                                                                   'date'] >= (
                                                                                           datetime.now() - relativedelta(
                                                                                       years=5)).strftime(
        '%Y-01-01')) & (OECD_Europe_utilisation_5yr_range['date'] <= (datetime.now() - relativedelta(years=1)).strftime(
        '%Y-12-01')) & ((pd.to_datetime(OECD_Europe_utilisation_5yr_range['date'])).dt.strftime('%Y') != '2020')]

    OECD_Europe_utilisation_5yr_range_max = OECD_Europe_utilisation_5yr_range[['utilisation', 'date']].groupby(
        pd.to_datetime(OECD_Europe_utilisation_5yr_range['date']).dt.month).max()

    OECD_Europe_utilisation_5yr_range_min = OECD_Europe_utilisation_5yr_range[['utilisation', 'date']].groupby(
        pd.to_datetime(OECD_Europe_utilisation_5yr_range['date']).dt.month).min()

    OECD_Europe_utilisation_forecast = pd.DataFrame(data=None, columns=run_forecast.columns)
    OECD_Europe_utilisation_forecast.loc['OECD Europe Utilisation Forecast'] = (
        run_forecast.sum(axis=0)).div(((df_available_capacity.drop(
        ['France'])).sum(axis=0)),axis=0)
    OECD_Europe_utilisation_forecast.replace(np.inf, 0, inplace=True)
    OECD_Europe_utilisation_forecast.replace(0, np.nan, inplace=True)

    OECD_Europe_utilisation_plot = [(datetime.now() - relativedelta(years=2)).strftime('%Y'),
                                    (datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                    (datetime.now()).strftime('%Y')]
    OECD_Europe_utilisation_plot[0] = (OECD_Europe_utilisation.loc[:,
                                       (pd.to_datetime(OECD_Europe_utilisation.columns)).strftime('%Y') ==
                                       OECD_Europe_utilisation_plot[0]]).copy()
    OECD_Europe_utilisation_plot[0].replace(0, np.nan, inplace=True)
    OECD_Europe_utilisation_plot[1] = (OECD_Europe_utilisation.loc[:,
                                       (pd.to_datetime(OECD_Europe_utilisation.columns)).strftime('%Y') ==
                                       OECD_Europe_utilisation_plot[1]]).copy()
    OECD_Europe_utilisation_plot[1].replace(0, np.nan, inplace=True)
    OECD_Europe_utilisation_plot[2] = (OECD_Europe_utilisation.loc[:,
                                       (pd.to_datetime(OECD_Europe_utilisation.columns)).strftime('%Y') ==
                                       OECD_Europe_utilisation_plot[2]]).copy()
    OECD_Europe_utilisation_plot[2].replace(0, np.nan, inplace=True)

    # prepare forecast for plot by limiting date to this year
    OECD_Europe_utilisation_forecast_plot = [(datetime.now() - relativedelta(years=1)).strftime('%Y'),
                                             (datetime.now()).strftime('%Y'),(datetime.now() + relativedelta(years=1)).strftime('%Y')]
    OECD_Europe_utilisation_forecast_plot[0] = (OECD_Europe_utilisation_forecast.loc[:,
                                                (pd.to_datetime(OECD_Europe_utilisation_forecast.columns)).strftime(
                                                    '%Y') == OECD_Europe_utilisation_forecast_plot[0]]).copy()
    OECD_Europe_utilisation_forecast_plot[1] = (OECD_Europe_utilisation_forecast.loc[:,
                                                (pd.to_datetime(OECD_Europe_utilisation_forecast.columns)).strftime(
                                                    '%Y') == OECD_Europe_utilisation_forecast_plot[1]]).copy()
    OECD_Europe_utilisation_forecast_plot[2] = (OECD_Europe_utilisation_forecast.loc[:,
                                                (pd.to_datetime(OECD_Europe_utilisation_forecast.columns)).strftime(
                                                    '%Y') == OECD_Europe_utilisation_forecast_plot[2]]).copy()

    # change column headers back to datetime in day of year format to allow all line on same graph
    OECD_Europe_utilisation_forecast_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_forecast_plot[0].columns)).dayofyear, format='%j')
    OECD_Europe_utilisation_forecast_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_forecast_plot[1].columns)).dayofyear, format='%j')
    OECD_Europe_utilisation_forecast_plot[2].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_forecast_plot[2].columns)).dayofyear, format='%j')
    OECD_Europe_utilisation_plot[0].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_plot[0].columns)).dayofyear, format='%j')
    OECD_Europe_utilisation_plot[1].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_plot[1].columns)).dayofyear, format='%j')
    OECD_Europe_utilisation_plot[2].columns = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_plot[2].columns)).dayofyear, format='%j')
    OECD_Europe_utilisation_5yr_range_max['date'] = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_5yr_range_max['date'])).dt.dayofyear, format='%j')
    OECD_Europe_utilisation_5yr_range_min['date'] = pd.to_datetime(
        (pd.to_datetime(OECD_Europe_utilisation_5yr_range_min['date'])).dt.dayofyear, format='%j')

    # Plot Regional Graph
    plt.style.use('seaborn-darkgrid')
    fig6, ax6 = plt.subplots(1, figsize=(15, 7.5))

    # sns.set_style('seaborn-dark-palette')
    ax6.plot(OECD_Europe_utilisation_plot[0].columns, OECD_Europe_utilisation_plot[0].loc['OECD Europe Utilisation'], marker='.', color='green',markerfacecolor='green', label=(datetime.now() - relativedelta(years=2)).strftime('%Y') + " Actual Utilisation")
    ax6.plot(OECD_Europe_utilisation_plot[1].columns, OECD_Europe_utilisation_plot[1].loc['OECD Europe Utilisation'], marker='.', color='violet',markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Actual Utilisation")
    ax6.plot(OECD_Europe_utilisation_plot[2].columns, OECD_Europe_utilisation_plot[2].loc['OECD Europe Utilisation'], marker='.', color='red',markerfacecolor='red', label=datetime.now().strftime('%Y') + " Actual Utilisation")
    ax6.plot(OECD_Europe_utilisation_forecast_plot[0].columns, OECD_Europe_utilisation_forecast_plot[0].loc['OECD Europe Utilisation Forecast'], marker='.', linestyle='dashed', color='violet', markerfacecolor='violet',label=(datetime.now() - relativedelta(years=1)).strftime('%Y') + " Forecast Utilisation")
    ax6.plot(OECD_Europe_utilisation_forecast_plot[1].columns, OECD_Europe_utilisation_forecast_plot[1].loc['OECD Europe Utilisation Forecast'],marker='.', linestyle='dashed', color='red', markerfacecolor='red',label=datetime.now().strftime('%Y') + " Forecast Utilisation")
#     ax6.plot(OECD_Europe_utilisation_forecast_plot[2].columns, OECD_Europe_utilisation_forecast_plot[2].loc['OECD Europe Utilisation Forecast'],marker='.', linestyle='dashed', color='sienna', markerfacecolor='sienna',label=(datetime.now() + relativedelta(years=1)).strftime('%Y') + " Forecast Utilisation")
    plt.fill_between(OECD_Europe_utilisation_5yr_range_max['date'].values,OECD_Europe_utilisation_5yr_range_min['utilisation'],OECD_Europe_utilisation_5yr_range_max['utilisation'],color='grey',alpha=0.2, label="5yr Range")


    ax6.set(ylabel="Utilisation Rate")
    major_format = mdates.DateFormatter('%b')
    ax6.xaxis.set_major_formatter(major_format)
    ax6.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax6.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=None, symbol='%', is_latex=False))
    plt.title('OECD Europe Utilisation Forecast')
    ax6.legend(loc='lower center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=4)
    fig6.subplots_adjust(bottom=0.6)
    fig6.tight_layout()
    fig6.savefig(img_subj6,bbox_inches='tight')
    plt.close()

### ------------------ create plot images (not regular script)-----------------------

In [4]:
# img_subj= os.path.join(folder_diagrams,'European_Run_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
# img_subj1= os.path.join(folder_diagrams,'NWE_Run_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
# img_subj2= os.path.join(folder_diagrams,'Med_Run_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
# img_subj3= os.path.join(folder_diagrams,'Maintenance_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
# img_subj4= os.path.join(folder_diagrams,'NWE_Utilisation_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
# img_subj5= os.path.join(folder_diagrams,'Med_Utilisation_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
# img_subj6= os.path.join(folder_diagrams,'European_Utilisation_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")


# get_regional_run_plot()
# get_regional_utilisation_plots()
# get_country_run_plots()
# get_country_utilisation_plots()
# get_maintenance_plot()

In [5]:
# margin(1,12,'nwecomplex')

### ------------------ Data frames processing -----------------------

In [12]:
nameplate_cap = raw_data_nameplate_cap().set_index('country')
print("Model Nameplate Capacity")
# print(nameplate_cap)
df_planned_known = raw_data_planned_known().set_index('country')
print("Model Planned Known")
# print(df_planned_known)
df_unplanned_known = raw_data_unplanned_known().set_index('country')
print("Model Unplanned Known")
# print(df_unplanned_known)
df_planned_avg = raw_data_planned_avg().set_index('country')
print("Model 5 year Planned Avg")
# print(df_planned_avg)
df_unplanned_avg = raw_data_unplanned_avg().set_index('country')
print("Model 5 year Unplanned Avg")
# print(df_unplanned_avg)
##IEA = raw_data_IEA().set_index('country')
##print("Latest IEA")
##print(IEA)
JODI = raw_data_JODI().set_index('country')
JODI.at['Greece','2022-02-01'] = 476.1 #override JODI runs
print("Model Latest JODI")
# print(JODI)
margin_table = raw_data_margin_table().set_index('region')
print("Model Forecast Margins")
# print(margin_table)

Model Nameplate Capacity


KeyboardInterrupt: 

In [13]:
# cor_nameplate_cap = correlation_data_cor_nameplate_cap().set_index('country')
# print("Correlation Nameplate Capacity")
# print(cor_nameplate_cap)
df_cor_planned_known = correlation_data_cor_planned_known().set_index('country')
print("Correlation Planned Known")
# print(df_cor_planned_known)
df_cor_unplanned_known = correlation_data_cor_unplanned_known().set_index('country')
print("Correlation Unplanned Known")
# print(df_cor_unplanned_known)
cor_available_cap = cor_nameplate_cap - df_cor_planned_known - df_cor_unplanned_known
cor_JODI = correlation_data_cor_JODI().set_index('country')
print("Correlation JODI")
# print(cor_JODI)
cor_margin_table = correlation_data_cor_margin_table().set_index('region')
print("Correlation Margins")
# print(cor_margin_table)

Correlation Nameplate Capacity
Correlation Planned Known
ERROR ON 2019-07-01
ERROR ON 2019-07-01
ERROR ON 2020-10-01
ERROR ON 2020-10-01
Error: C:\Users\syedahmad\Documents\refinery_runs_model\runs_model\2022-10-11 file not found
Correlation Unplanned Known


KeyboardInterrupt: 

In [16]:
df_cor_unplanned_known

2017-01-01  2017-02-01  2017-03-01  2017-04-01  2017-05-01  2017-06-01  2017-07-01  2017-08-01  2017-09-01  2017-10-01  2017-11-01  2017-12-01  2018-01-01  2018-02-01  2018-03-01  2018-04-01  2018-05-01  2018-06-01  2018-07-01  2018-08-01  2018-09-01  2018-10-01  2018-11-01  2018-12-01  2019-01-01  2019-02-01  2019-03-01  2019-04-01  2019-05-01  2019-06-01  2019-07-01  2019-08-01  2019-09-01  2019-10-01  2019-11-01  2019-12-01  2020-01-01  2020-02-01  2020-03-01  2020-04-01  2020-05-01  2020-06-01  2020-07-01  2020-08-01  2020-09-01  2020-10-01  2020-11-01  2020-12-01  2021-01-01  2021-02-01  2021-03-01  2021-04-01  2021-05-01  2021-06-01  2021-07-01  2021-08-01  2021-09-01  2021-10-01  2021-11-01  2021-12-01  2022-01-01  2022-02-01  2022-03-01  2022-04-01  2022-05-01
country                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Austria               0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00
Belgium               0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00       11.79        0.00       55.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00       34.10        0.00        0.00        0.00        7.33        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        3.55        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00       13.61       21.10        0.00        0.00        0.00        0.00        0.00        0.00
Czech Republic        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00       14.28        5.00        0.00        1.07        6.27        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        2.19       29.47        0.00        0.00        0.00        0.00        0.00        0.00        0.00       32.90        0.00        0.00        0.00        0.00        0.00       31.98        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00
Denmark               0.00        0.00   

In [ ]:
df_planned_forecast = get_planned_outage_forecast()
print("Full Planned TAR Forecast")
# print(df_planned_forecast)
df_planned_forecast_which_are_known = get_planned_outage_forecast_which_are_known()
print("Full Planned TAR Forecast of which are Known")
# print(df_planned_forecast_which_are_known)
df_planned_forecast_which_are_avg = get_planned_outage_forecast_which_are_avg()
print("Full Planned TAR Forecast of which are 5 yr Averages")
# print(df_planned_forecast_which_are_avg)
df_unplanned_forecast = get_unplanned_outage_forecast()
print("Full Unplanned Outage Forecast")
# print(df_unplanned_forecast)
df_unplanned_forecast_which_are_known = get_unplanned_outage_forecast_which_are_known()
print("Full Unplanned Outage Forecast of which are Known")
# print(df_unplanned_forecast_which_are_known)
df_unplanned_forecast_which_are_avg = get_unplanned_outage_forecast_which_are_avg()
print("Full Unplanned Outage Forecast of which are 5 yr Averages")
# print(df_unplanned_forecast_which_are_avg)

df_available_capacity = get_available_capacity()
print('Available Capacity')
# print(df_available_capacity)

In [ ]:
correlation_params = get_correlation()
print("Correlation Parameters for each Country")
# print(correlation_params)

utilisation_forecast = get_utilisation_forecast()
# print(utilisation_forecast)

run_forecast= get_run_forecast().drop(['France'])
print('Run Forecast')
# print(run_forecast)

max_range=get_max_range().set_index('index')
min_range=get_min_range().set_index('index')

last_week_forecast=get_runs_forecast()

### ------------------ create scraper file -----------------

In [17]:
with pd.option_context('display.multi_sparse',False):
    scrape_run = run_forecast.stack()
    scrape_run = (scrape_run.to_frame()).reset_index()
    scrape_run.columns = ['COUNTRY', 'FORECAST_DATE', 'FORECAST_RUNS']

    scrape_utilisation = utilisation_forecast.stack()
    scrape_utilisation = (scrape_utilisation.to_frame()).reset_index()
    scrape_utilisation.columns = ['COUNTRY', 'FORECAST_DATE', 'FORECAST_UTILISATION']

    scrape_planned_off = df_planned_forecast.drop(['France'])
    scrape_planned_off = scrape_planned_off.stack()
    scrape_planned_off = (scrape_planned_off.to_frame()).reset_index()
    scrape_planned_off.columns = ['COUNTRY', 'FORECAST_DATE', 'FORECAST_PLANNED_OFF']

    scrape_unplanned_off = df_unplanned_forecast.drop(['France'])
    scrape_unplanned_off = scrape_unplanned_off.stack()
    scrape_unplanned_off = (scrape_unplanned_off.to_frame()).reset_index()
    scrape_unplanned_off.columns = ['COUNTRY', 'FORECAST_DATE', 'FORECAST_UNPLANNED_OFF']

    scrape_available_capacity = df_available_capacity.drop(['France'])
    scrape_available_capacity = scrape_available_capacity.stack()
    scrape_available_capacity = (scrape_available_capacity.to_frame()).reset_index()
    scrape_available_capacity.columns = ['COUNTRY', 'FORECAST_DATE', 'FORECAST_AVAILABLE_CAPACITY']

    scrape = pd.merge(scrape_run,scrape_utilisation,how='inner',on= ["COUNTRY","FORECAST_DATE"])
    for subject in [scrape_planned_off,scrape_unplanned_off,scrape_available_capacity]:
        scrape = pd.merge(scrape, subject, how='inner', on=["COUNTRY", "FORECAST_DATE"])
    scrape['PDATE'] = (datetime.now()).strftime('%Y-%m-%d')
    scrape['WORLD_REG'] = 'Europe'
    scrape['UNITS'] = 'KBD'
    scrape['SUB_REGION'] = np.where((scrape['COUNTRY'].isin(NWE_Countries)), 'NWE','Med')
    scrape["FORECAST_RUNS"] = pd.to_numeric(scrape["FORECAST_RUNS"])
    scrape["FORECAST_UTILISATION"] = pd.to_numeric(scrape["FORECAST_UTILISATION"])*100
    scrape=scrape.round(2)
    scrape.to_csv(os.path.join(folder_scrape, 'Upload_OIL_RefineryCountryRunsForecast-'+ datetime.today().strftime('%y-%m-%d') + '.csv'), index=False)

    df_pivot_runs = pd.pivot_table(scrape, values='FORECAST_RUNS', columns=['FORECAST_DATE'], index=['COUNTRY'])
    df_pivot_utilisation = pd.pivot_table(scrape, values='FORECAST_UTILISATION', columns=['FORECAST_DATE'], index=['COUNTRY'])
    df_pivot_planned_offline = pd.pivot_table(scrape, values='FORECAST_PLANNED_OFF', columns=['FORECAST_DATE'], index=['COUNTRY'])
    df_pivot_unplanned_offline = pd.pivot_table(scrape, values='FORECAST_UNPLANNED_OFF', columns=['FORECAST_DATE'], index=['COUNTRY'])
    df_pivot_available_capacity = pd.pivot_table(scrape, values='FORECAST_AVAILABLE_CAPACITY', columns=['FORECAST_DATE'],index=['COUNTRY'])
    df_pivot_nameplate_capacity = nameplate_cap.drop(['France'])
    df_pivot_planned_known_offline = df_planned_forecast_which_are_known.drop(['France'])
    df_pivot_planned_avg_offline = df_planned_forecast_which_are_avg.drop(['France'])
    df_pivot_unplanned_known_offline = df_unplanned_forecast_which_are_known.drop(['France'])
    df_pivot_unplanned_avg_offline = df_unplanned_forecast_which_are_avg.drop(['France'])



    writer = pd.ExcelWriter(os.path.join(folder_results, 'Country_Runs_Forecast_Results_'+ datetime.today().strftime('%Y-%m-%d') + '.xlsx'), engine='xlsxwriter')

    df_pivot_runs.to_excel(writer, sheet_name='Runs Forecast')
    df_pivot_utilisation.to_excel(writer, sheet_name='Utilisation Forecast')
    df_pivot_planned_offline.to_excel(writer, sheet_name='Planned TAR Forecast')
    df_pivot_unplanned_offline.to_excel(writer, sheet_name='Unplanned Outage Forecast')
    df_pivot_available_capacity.to_excel(writer, sheet_name='Available Capacity Forecast')
    df_pivot_nameplate_capacity.to_excel(writer, sheet_name='Nameplate Capacity Forecast')
    df_pivot_planned_known_offline.to_excel(writer, sheet_name='Planned Known TAR Forecast')
    df_pivot_planned_avg_offline.to_excel(writer, sheet_name='Planned TAR Avg Forecast')
    df_pivot_unplanned_known_offline.to_excel(writer, sheet_name='Unplanned Known Outage Forecast')
    df_pivot_unplanned_avg_offline.to_excel(writer, sheet_name='Unplanned Outage Avg Forecast')
    writer.save()


In [18]:
#Joined Tables (future and historic)
nameplate_capacity_join = pd.concat([cor_nameplate_cap, nameplate_cap], axis=1, join="inner")
planned_forecast_which_are_known_join = pd.concat([df_cor_planned_known, df_planned_known], axis=1, join="inner")
unplanned_forecast_which_are_known_join = pd.concat([df_cor_unplanned_known, df_unplanned_known], axis=1, join="inner")
cor_JODI.loc['France Med'] = cor_JODI.loc['France'] * ((cor_available_cap.loc['France Med']) / (cor_available_cap.loc['France']))
cor_JODI.loc['France NWE'] = cor_JODI.loc['France'] * ((cor_available_cap.loc['France NWE']) / (cor_available_cap.loc['France']))
JODI.loc['France Med'] = JODI.loc['France'] * ((df_available_capacity.loc['France Med']) / (df_available_capacity.loc['France']))
JODI.loc['France NWE'] = JODI.loc['France'] * ((df_available_capacity.loc['France NWE']) / (df_available_capacity.loc['France']))
JODI_join = pd.concat([cor_JODI, JODI], axis=1, join="inner")

nameplate_capacity_join = nameplate_capacity_join.loc[:,~nameplate_capacity_join.columns.duplicated()]
planned_forecast_which_are_known_join = planned_forecast_which_are_known_join.loc[:,~planned_forecast_which_are_known_join.columns.duplicated()]
unplanned_forecast_which_are_known_join = unplanned_forecast_which_are_known_join.loc[:,~unplanned_forecast_which_are_known_join.columns.duplicated()]
JODI_join = JODI_join.loc[:,~JODI_join.columns.duplicated()]

#get componentes for graphs (i.e delete countries i dont want and get the correct start date)
JODI_plot = JODI_join.drop(['France'])
JODI_plot = JODI_plot.loc[:, JODI_plot.columns >= (graph_start_date)]
#print(JODI_plot)

img_subj= os.path.join(folder_diagrams,'European_Run_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
img_subj1= os.path.join(folder_diagrams,'NWE_Run_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
img_subj2= os.path.join(folder_diagrams,'Med_Run_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
img_subj3= os.path.join(folder_diagrams,'Maintenance_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
img_subj4= os.path.join(folder_diagrams,'NWE_Utilisation_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
img_subj5= os.path.join(folder_diagrams,'Med_Utilisation_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")
img_subj6= os.path.join(folder_diagrams,'European_Utilisation_Forecast_'+ datetime.today().strftime('%y-%m-%d') + ".png")


get_regional_run_plot()
get_regional_utilisation_plots()
get_country_run_plots()
get_country_utilisation_plots()
get_maintenance_plot()


In [19]:
def get_runs_table():
    df = run_forecast
#     del df.index.name
    df.loc['European Total'] = df.sum(axis=0)
    df.loc['European Other'] = df.loc['European Total'] - df.loc[
        ['Belgium', 'France NWE', 'France Med', 'Germany', 'Greece', 'Italy', 'Netherlands', 'Poland', 'Spain',
         'Turkey', 'United Kingdom']].sum(axis=0)
    df.loc['NWE'] = df.loc[NWE_Countries].sum(axis=0)
    df.loc['Med'] = df.loc[Med_Countries_for_table].sum(axis=0)
    df.loc['European Available Capacity'] = (df_available_capacity.drop(['France'])).sum(axis=0)
    df.loc['European Utilisation %'] = df.loc['European Total'].div(df.loc['European Available Capacity'],axis=0)*100
    df = df.T.reset_index()
    df = df[pd.to_datetime(df['index']).dt.year <= datetime.now().year]
    df['index'] = pd.to_datetime(df['index']).dt.strftime('%b-%y')
    df = df.set_index('index')
#     del df.index.name
    df['m-o-m change kb/d'] = df['European Total'].diff(periods=1)
    df['m-o-m change kb/d'] = df['m-o-m change kb/d'].astype(float).round()
    df = df[['Belgium','France NWE','France Med','Germany','Greece','Italy','Netherlands','Poland','Spain','Turkey','United Kingdom','European Other','NWE','Med','European Total','m-o-m change kb/d','European Utilisation %']]

    def color_negative_red(val):
        if val < 0:
            color = 'red'
        elif val > 0:
            color = 'green'
        else:
            color = 'black'
        return 'color: %s' % color
    cm = matplotlib.cm.get_cmap('Reds_r')
    df = df.apply(pd.to_numeric, errors='ignore')
    df = df.style.set_table_styles([
        {'selector': 'th',
         'props': [
             ('border-style', 'solid'),
             ('border-color', 'black'),
             ('vertical-align', 'top'),
             ('font-size', '10pt'), ('background-color', '#edeeef'), ('border-width', '1px'),
             ('border-collapse', 'collapse'), ('text-align', 'center'),('width','100px')
         ]
         }]
    ).set_properties(
        **{'background-color': 'white','font-size': '10pt', 'border-color': 'black', 'border-style': 'solid',
           'border-width': '1px', 'border-collapse': 'collapse', 'text-align': 'center'}).background_gradient(cmap=cm, subset=['Belgium','France NWE','France Med','Germany','Greece','Italy','Netherlands','Poland','Spain','Turkey','United Kingdom','European Other','NWE','Med','European Total','European Utilisation %']).applymap(color_negative_red,subset=pd.IndexSlice[:, ['m-o-m change kb/d']]).format('{:,.0f}').format(lambda x:  'NaN' if np.isnan(x) else '{:+g}'.format(x),subset=['m-o-m change kb/d']).render()
    return df

def get_maintenance_table():
    planned_known_total = df_planned_forecast_which_are_known.copy().drop(['France'])
    planned_avg_total = df_planned_forecast_which_are_avg.copy().drop(['France'])
    unplanned_known_total = df_unplanned_forecast_which_are_known.copy().drop(['France'])
    unplanned_avg_total = df_unplanned_forecast_which_are_avg.copy().drop(['France'])
    total_maintenance = planned_known_total + planned_avg_total +unplanned_known_total + unplanned_avg_total

    df_maintenance_total = pd.DataFrame(columns= planned_known_total.columns)
    df_maintenance_total=df_maintenance_total.append(total_maintenance)
    df_maintenance_total.loc['European Other'] = total_maintenance.sum(axis=0) - total_maintenance.loc[
        ['Belgium', 'France NWE', 'France Med', 'Germany', 'Greece', 'Italy', 'Netherlands', 'Poland', 'Spain',
         'Turkey', 'United Kingdom']].sum(axis=0)
    df_maintenance_total.loc['NWE'] = total_maintenance.loc[NWE_Countries].sum(axis=0)
    df_maintenance_total.loc['Med'] = total_maintenance.loc[Med_Countries_for_table].sum(axis=0)
    df_maintenance_total.loc['Planned TARs known kb/d'] = planned_known_total.sum(axis=0)
    df_maintenance_total.loc['Planned TAR placeholders kb/d'] = planned_avg_total.sum(axis=0)
    df_maintenance_total.loc['Unplanned outages known kb/d'] = unplanned_known_total.sum(axis=0)
    df_maintenance_total.loc['Unplanned outage placeholders kb/d'] = unplanned_avg_total.sum(axis=0)
    df_maintenance_total.loc['Total offline capacity kb/d'] = total_maintenance.sum(axis=0)
    df_maintenance_total = df_maintenance_total.T.reset_index()
#     df_maintenance_total = df_maintenance_total[pd.to_datetime(df_maintenance_total['index']).dt.year <= datetime.now().year]
    df_maintenance_total = df_maintenance_total[pd.to_datetime(df_maintenance_total['index']) <= (datetime.now() + relativedelta(months=3))]
    df_maintenance_total['index'] = pd.to_datetime(df_maintenance_total['index']).dt.strftime('%b-%y')
    df_maintenance_total = df_maintenance_total.set_index('index')
#     del df_maintenance_total.index.name
    df_maintenance_total['m-o-m change kb/d'] = df_maintenance_total['Total offline capacity kb/d'].diff(periods=1)
    df_maintenance_total['m-o-m change kb/d'] = df_maintenance_total['m-o-m change kb/d'].astype(float).round()
    df_maintenance_total = df_maintenance_total[['Belgium','France NWE','France Med','Germany','Greece','Italy','Netherlands','Poland','Spain','Turkey','United Kingdom','European Other','NWE','Med','Planned TARs known kb/d','Planned TAR placeholders kb/d','Unplanned outages known kb/d','Unplanned outage placeholders kb/d','Total offline capacity kb/d','m-o-m change kb/d']]

    def color_negative_red(val):
        if val < 0:
            color = 'red'
        elif val > 0:
            color = 'green'
        else:
            color = 'black'
        return 'color: %s' % color

    cm = matplotlib.cm.get_cmap('Reds')
    df_maintenance_total = df_maintenance_total.apply(pd.to_numeric, errors='ignore')
    df = df_maintenance_total.style.set_table_styles([
    {'selector': 'th',
     'props': [
         ('border-style', 'solid'),
         ('border-color', 'black'),
         ('vertical-align', 'top'),
         ('font-size', '10pt'), ('background-color', '#edeeef'), ('border-width', '1px'),
         ('border-collapse', 'collapse'), ('text-align', 'center'), ('width', '100px')]}]).set_properties(
    **{'background-color': 'white', 'font-size': '10pt', 'border-color': 'black', 'border-style': 'solid',
       'border-width': '1px', 'border-collapse': 'collapse', 'text-align': 'center'}).background_gradient(cmap=cm,axis=0,
                                                                                                           subset=['Belgium','France NWE','France Med','Germany','Greece','Italy','Netherlands','Poland','Spain','Turkey','United Kingdom','European Other','NWE','Med','Total offline capacity kb/d']).format('{:,.0f}').applymap(color_negative_red,subset=pd.IndexSlice[:, ['m-o-m change kb/d']]).format('{:,.0f}').format(lambda x:  'NaN' if np.isnan(x) else '{:+g}'.format(x),subset=['m-o-m change kb/d']).render()
    return df


def get_closures_list():
    query = ''' SELECT distinct
              RefineryUnit.PLANT_NAME AS 'Plant Name',[EndDate] As 'Closure Date'
              ,sum([Capacity])/1000 AS 'Capacity kbpd'

          FROM [OIL].[Refinery].[RefineryUnitCapacitySchedule],[Refinery].RefineryUnit,[Refinery].RefineryPlant
          where [RefineryUnitCapacitySchedule].Unit_ID = [RefineryUnit].Unit_ID
          and [RefineryUnit].PLANT_ID = RefineryPlant.PLANT_ID
          and EndDate != '9998-12-31'
          and EndDate >= '2020-01-01'
          and [RefineryUnit].UTYPE_DESC = 'Atmospheric Distillation'
          AND [RefineryUnit].[COUNTRY] IN (''' + list_to_string(IIR_Countries) + ''')
          group by RefineryUnit.PLANT_NAME,[EndDate]
            '''
    df = ag_get_data(query)
    df['Capacity kbpd'] = df['Capacity kbpd'].astype(int)
    df['Closure Date'] = (pd.to_datetime(df['Closure Date'])).dt.strftime('%Y-%m-%d')

    df = df[['Plant Name', 'Capacity kbpd', 'Closure Date']]
    df = df.style.set_table_styles([
        {'selector': 'th',
         'props': [
             ('border-style', 'solid'),
             ('border-color', 'black'),
             ('vertical-align', 'top'),
             ('font-size', '10pt'), ('background-color', '#edeeef'), ('border-width', '1px'),
             ('border-collapse', 'collapse'), ('text-align', 'center'),('width','100px')
         ]
         }]
    ).set_properties(
        **{'font-size': '10pt', 'background-color': '#edeeef', 'border-color': 'black', 'border-style': 'solid',
           'border-width': '1px', 'border-collapse': 'collapse', 'text-align': 'center'}).hide_index().render()
    return df

def get_maintenance_list():
    def highlight_col(x):
        # copy df to new - original data are not changed
        df_copy = df.copy()
        # set by condition
        mask = df_copy['Status'] == 'Ongoing'
        df_copy.loc[mask, :] = 'background-color: yellow'
        df_copy.loc[~mask, :] = 'background-color: ""'
        return df_copy

    query = '''WITH temp0 AS
    (
                    SELECT DISTINCT TOP (''' + Pdate_days_before + ''') ([MaintenanceDaily].[pdate])
                    FROM            [Refinery].[MaintenanceDaily]
                    ORDER BY        [PDATE] DESC)
    SELECT DISTINCT [RefineryMaintenanceBestLatest].[Modified],
                    [RefineryUnit].[COUNTRY]                          AS Country,
                    [RefineryUnit].[PLANT_NAME]                       AS 'Plant Name',
                    [RefineryMaintenanceBestLatest].[UNIT_NAME]       AS 'Unit Name',
                    [RefineryMaintenanceBestLatest].[U_CAPACITY] /1000 AS 'Nameplate Capacity kbpd' ,
                    [RefineryMaintenanceBestLatest].[CAP_OFFLINE]/1000 AS 'Capacity Offline kbpd',
                    [RefineryMaintenanceBestLatest].[START_DATE]       AS 'Start Date' ,
                    [RefineryMaintenanceBestLatest].[END_DATE]         AS 'End Date',
                    [RefineryMaintenanceBestLatest].[EVENT_TYPE]       AS 'Type' ,
                    [RefineryMaintenanceBestLatest].[E_CAUSE]          AS 'Cause'
    FROM            [Refinery].[refineryunit],
                    [Refinery].[RefineryMaintenanceBestLatest],
                    [Refinery].[RefineryPlant],
                    [Refinery].[MaintenanceDaily]
    WHERE           [RefineryPlant].[PLANT_ID] = [RefineryUnit].[PLANT_ID]
    AND             [RefineryMaintenanceBestLatest].[unit_id] = [refineryunit].[unit_id]
    AND             [MaintenanceDaily].[EVENT_ID] = [RefineryMaintenanceBestLatest].[EVENT_ID]
    AND             [refineryunit].[utype_desc] = 'Atmospheric Distillation'
    AND             [RefineryMaintenanceBestLatest].[E_STATUS] <> 'Cancelled'
    AND             [RefineryUnit].[COUNTRY] IN (''' + list_to_string(IIR_Countries) + ''')
    AND             ((
                                                    [MaintenanceDaily].[outage_date] >= ''' + "'" + (
        datetime.now()).strftime('%Y-01-01') + "'" + '''
                                    AND             [MaintenanceDaily].[outage_date] < ''' + "'" + (
                        datetime.now() + relativedelta(years=2)).strftime('%Y-01-01') + "'" + ''' ))
    AND             (
                                    [refinerymaintenancebestlatest].[DERATE] IS NULL
                    OR              ((
                                                                    [refinerymaintenancebestlatest].[E_CAUSE] = 'Electrical'
                                                    OR              [refinerymaintenancebestlatest].[E_CAUSE] = 'Mechanical'
                                                    OR              [refinerymaintenancebestlatest].[E_CAUSE] = 'Weather'
                                                    OR              [refinerymaintenancebestlatest].[E_CAUSE] IS NULL)
                                    AND             [refinerymaintenancebestlatest].[DERATE] = 'DERATE'))
    AND             [MaintenanceDaily].[pdate] =
                    (
                             SELECT TOP(1)
                                      [pdate]
                             FROM     temp0
                             ORDER BY [PDATE])
    ORDER BY        [refineryunit].[country],
                    [RefineryUnit].[PLANT_NAME],
                    [RefineryMaintenanceBestLatest].[START_DATE],
                    [RefineryMaintenanceBestLatest].[END_DATE],
                    [RefineryMaintenanceBestLatest].[Modified] DESC

        '''
    df = ag_get_data(query)
    df = df.reset_index()
    df = df.round(1)
    df['Modified'] = (pd.to_datetime(df['Modified']))
    df['Start Date'] = (pd.to_datetime(df['Start Date']))
    df['End Date'] = (pd.to_datetime(df['End Date']))
    df = df[['Country', 'Plant Name', 'Unit Name', 'Start Date', 'End Date', 'Nameplate Capacity kbpd',
             'Capacity Offline kbpd', 'Cause', 'Type', 'Modified']]

    annual_timeframe = [pd.date_range(*r, freq='D') for r in df[['Start Date', 'End Date']].values]
    annual_timeframe = pd.DataFrame(annual_timeframe)

    df = df[['Country', 'Plant Name', 'Unit Name', 'Start Date', 'End Date', 'Nameplate Capacity kbpd',
             'Capacity Offline kbpd', 'Cause', 'Type', 'Modified']].join(annual_timeframe).set_index(
        ['Country', 'Plant Name', 'Unit Name', 'Start Date', 'End Date', 'Nameplate Capacity kbpd',
         'Capacity Offline kbpd', 'Cause', 'Type','Modified']).stack().reset_index().drop('level_10', axis=1)
    df.columns = ['Country', 'Plant Name', 'Unit Name', 'Start Date', 'End Date', 'Nameplate Capacity kbpd',
                  'Capacity Offline kbpd', 'Cause', 'Type', 'Modified', 'Date']
    df['Date'] = (pd.to_datetime(df['Date'])).dt.strftime('%Y-%m')
    df = df.drop_duplicates()
#     df = df[pd.to_datetime(df['Date']).dt.year == datetime.now().year]
#     df = df[pd.to_datetime(df['Date']).dt.month >= (datetime.now() - relativedelta(months=1)).month]
    df = df[pd.to_datetime(df['Date']) >= pd.to_datetime((datetime.now()).strftime('%Y-%m-01'))]
    df = df[pd.to_datetime(df['Date']) <= (datetime.now() + relativedelta(months=2))]
    df = df.sort_values(by=['Date', 'Country', 'Plant Name'])
    df['Date'] = (pd.to_datetime(df['Date'])).dt.strftime('%b-%y')
#     df = df.reset_index(drop=True).set_index(['Date', 'Country', 'Plant Name', 'Unit Name'])
#     df['Modified'] = (pd.to_datetime(df['Modified'])).dt.strftime('%Y-%m-%d')
    df = df.drop(columns=['Modified'])
    #df = df.reset_index(drop=True).set_index(['Date', 'Country', 'Plant Name', 'Unit Name', 'Type'])
    df['Start Date'] = (pd.to_datetime(df['Start Date'])).dt.strftime('%Y-%m-%d')
    df['End Date'] = (pd.to_datetime(df['End Date'])).dt.strftime('%Y-%m-%d')
#     df = df.drop(columns=['Modified'])
#     df = df.style.set_table_styles([
    df['Nameplate Capacity kbpd'] = df['Nameplate Capacity kbpd'].astype(int)
    df['Capacity Offline kbpd'] = df['Capacity Offline kbpd'].astype(int)
    df = df.reset_index(drop=True).style.set_table_styles([
        {'selector': 'th',
         'props': [
             ('border-style', 'solid'),
             ('border-color', 'black'),
             ('vertical-align', 'top'),
             ('font-size', '10pt'), ('background-color', '#edeeef'), ('border-width', '1px'),
             ('border-collapse', 'collapse'), ('text-align', 'center'),('width','100px')
         ]
         }]
    ).set_properties(
        **{'font-size': '10pt', 'background-color': '#edeeef', 'border-color': 'black', 'border-style': 'solid',
           'border-width': '1px', 'border-collapse': 'collapse', 'text-align': 'center'}).render()
    return df

In [20]:
def send_report(email_from, emails, img):
    #format_datetime = '%y-%m-%d'
    folder, filename = os.path.split(img_subj)
    folder, filename1 = os.path.split(img_subj1)
    folder, filename2 = os.path.split(img_subj2)
    folder, filename3 = os.path.split(img_subj3)
    folder, filename4 = os.path.split(img_subj4)
    folder, filename5 = os.path.split(img_subj5)
    folder, filename6 = os.path.split(img_subj6)


    sub = "European Refinery Run Forecast" #- (" + datetime.today().strftime(format_datetime) + ")"
    body = "<font size='+1'><b>1.OECD Europe Refinery Run Forecast.</b></font>" \
           + "<br>" \
           + "<br>" \
           + "<img src='cid:" + filename + "'>" \
           + "<br>" \
           + "<br>" \
           + "<font><b>1.1.Crude runs forecast table.<b></font>" \
           + "<br>" \
           + "<font>Please note that the utilisation rate is the run rate of operable capacity (as opposed to nameplate capacity).</font>" \
           + "<br>" \
           + "<font>Conditional formatting below indicates where runs are the highest/lowest with white/red respectively within the timeframe shown.</font>" \
           + "<br>" \
           + get_runs_table() \
           + "<br>" \
           + "<br>" \
           + "<font size='+1'><b>2.OECD Europe Maintenance Forecast.</b></font>" \
           + "<br>" \
           + "<font>Individual maintenance events of forward looking months can be found at the end of the report.</font>" \
           + "<br>" \
           + "<img src='cid:" + filename3 + "'>" \
           + "<br>" \
           + "<br>" \
           + "<font><b>2.1.European maintenance forecast table.<b></font>" \
           + "<br>" \
           + "<font>Conditional formatting below indicates where runs are the highest/lowest with red/white respectively within the timeframe shown.</font>" \
           + "<br>" \
           + get_maintenance_table() \
           + "<br>" \
           + "<br>" \
           + "<font size='+1'><b>3.Implied OECD Europe Utilisation Forecast. </b></font>" \
           + "<br>" \
           + "<font>Utilisation rate is calculated using the JODI/forecast runs over our view of operable capacity.</font>" \
           + "<br>" \
           + "<img src='cid:" + filename6 + "'>" \
           + "<br>" \
           + "<br>" \
           + "<br>" \
           + "<font size='+1'><b>4.Assumptions.</b></font>" \
           + "<br>" \
           + "<br>" \
           + "<font><b>4.1.Closures and Mothballed Units.<b></font>" \
           + "<br>" \
           + "<br>" \
           + get_closures_list() \
           + "<br>" \
           + "<font><b>4.2.Maintenance List.<b></font>" \
           + "<br>" \
           + "<br>" \
           + get_maintenance_list() \
           + "<br>"
    email_sender = ag_email_utils.AgEmailHelper()
    email_sender.send_mail_with_embedded_images(email_from, emails, sub, body, img)

img=[img_subj,img_subj1,img_subj2,img_subj3,img_subj4,img_subj5,img_subj6]
send_report(sender_address_for_auto_email, recipient_address_for_auto_email, img)

end_time=time.time()
run_duration = (end_time-start_time)/60
print(run_duration)

69.97409706115722
